# 日本語の解析(形態素解析)について

## 形態素解析について  
形態素解析(Morphological Analysis)とは、自然言語の文章を、意味を持つ最小の単位である「形態素」に分割し、品詞を判別する作業。系サイト解析は、機械翻訳やかな漢字変換、また、テキストマイニングなど、様々な分野で利用されている。

## 代表的な形態素解析のライブラリ

### 利用実績の多い定番ライブラリ「MeCab」  
MeCabはCRF(Conditional Random Fields)を用いたパラメータ推定による形態素解析エンジンである。判別精度に加えて、実行速度が速いことも特徴である。

In [1]:
import MeCab

In [11]:
#mecab = MeCab.Tagger("-Ochasen") # MeCabオブジェクトを生成
mecab = MeCab.Tagger("") # MeCabオブジェクトを生成
malist = mecab.parse("庭には二羽鶏がいる。") # 形態素解析を行う
print(malist)

庭	ニワ	ニワ	庭	名詞-普通名詞-一般			0
に	ニ	ニ	に	助詞-格助詞			
は	ワ	ハ	は	助詞-係助詞			
二	フタ	フタ	二	名詞-数詞			2
羽	ワ	ワ	羽	接尾辞-名詞的-助数詞			
鶏	ニワトリ	ニワトリ	鶏	名詞-普通名詞-一般			0
が	ガ	ガ	が	助詞-格助詞			
いる	イル	イル	居る	動詞-非自立可能	上一段-ア行	終止形-一般	0
。			。	補助記号-句点			
EOS



### インストールがとにかく簡単なピュアPython解析器「Janome」

In [13]:
from janome.tokenizer import Tokenizer

In [15]:
t = Tokenizer()
malist = t.tokenize("庭には二羽鶏がいる。")
for n in malist:
    print(n)

庭	名詞,一般,*,*,*,*,庭,ニワ,ニワ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
二	名詞,数,*,*,*,*,二,ニ,ニ
羽	名詞,接尾,助数詞,*,*,*,羽,ワ,ワ
鶏	名詞,一般,*,*,*,*,鶏,ニワトリ,ニワトリ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
いる	動詞,自立,*,*,一段,基本形,いる,イル,イル
。	記号,句点,*,*,*,*,。,。,。


## 出現頻度の解析

In [16]:
from janome.tokenizer import Tokenizer
import zipfile
import os.path, urllib.request as req

In [17]:
# 銀河鉄道の夜のzipファイルをダウンロード
url = "http://www.aozora.gr.jp/cards/000081/files/456_ruby_145.zip"
local = "456_ruby_145.zip"
if not os.path.exists(local):
    print("zipファイルをダウンロード")
    req.urlretrieve(url, local)

zipファイルをダウンロード


In [18]:
# zipファイル内のテキストファイルを読む
zf = zipfile.ZipFile(local, 'r') # zipファイルを読む
fp = zf.open('gingatetsudono_yoru.txt', 'r') # アーカイブ内のテキストを読む
bindata = fp.read()
txt = bindata.decode('shift_jis') # テキストがShift_JISなのでデコード

In [21]:
# 形態素解析オブジェクトの生成
t = Tokenizer()

In [24]:
# テキストを一行ずつ処理
word_dic = {}
lines = txt.split("\r\n")
for line in lines:
    malist = t.tokenize(line)
    for w in malist:
        word = w.surface
        ps = w.part_of_speech # 品詞
        if ps.find('名詞') < 0:
            continue # 名詞だけカウント
        if not word in word_dic:
            word_dic[word] = 0
        word_dic[word] += 1

In [27]:
# よく使われる単語を表示
keys = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
for word, cnt in keys[:50]:
    print("{0}({1})".format(word, cnt), end="")

よう(209)の(206)ジョバンニ(190)人(102)カムパネルラ(101)ん(101)方(66)中(65)ぼく(59)それ(58)たち(56)みんな(54)二(53)一(47)何(46)ほんとう(45)鳥(45)どこ(43)こと(39)窓(39)汽車(39)前(38)そう(38)いま(38)眼(37)川(35)とき(35)僕(33)もの(32)たくさん(30)水(29)お(29)ら(29)青年(28)銀河(27)こっち(27)さっき(27)上(26)そこ(26)ろ(26)風(26)星(25)顔(25)向う(25)女の子(25)野原(24)お父さん(23)天の川(23)声(22)い(21)

# Word2Vecで文章をベクトル変換しよう

## Word2Vecについて  
「Word2Vec」は、文章中の語句をベクトルに変換するツールである。単語同士のつながりにも続いて単語同士の関係性をベクトル化する。その名の通り、単語をベクトルとして表現することで、その単語の意味をとらえる手法である。  
また、Word2Vecを使うと、意味の線形計算が可能になる。  
Word2Vecの代表的な実装手法には、Skip-gramやCBoWなどがある。Skip-gramでは、ある単語が与えられたとき、その単語の周辺の単語を学習しておいて、その単語の次にくる単語の出現率を計算することでモデル化を行う。

## GensimのWord2Vecで「こころ」を読む

In [1]:
from janome.tokenizer import Tokenizer
from gensim.models import word2vec
import re

In [2]:
# テキストファイルの読み込み
bindata = open('kokoro.txt.sjis', 'rb').read()
text = bindata.decode('shift_jis')

In [3]:
# テキストの先頭にあるヘッダーとフッターを削除
text = re.split(r'\-{5,}', text)[2]
text = re.split('底本：', text)[0]
text = text.strip()

In [4]:
# 形態素解析
t = Tokenizer()
results = []
# テキストを一行ずつ処理する
lines = text.split("\r\n")
for line in lines:
    s = line
    s = s.replace('|', '')
    s = re.sub(r'《.+?》', '', s) # ルビを削除
    s = re.sub(r'[#.+?]', '', s) # 入力注を削除
    tokens = t.tokenize(s) # 形態素解析
    # 必要な語句だけを対象とする
    r = []
    for tok in tokens:
        if tok.base_form == "*": # 単語の基本形を採用
            w = tok.surface
        else:
            w = tok.base_form
        ps = tok.part_of_speech # 品詞情報
        hinsi = ps.split(',')[0]
        if hinsi in ['名詞', '形容詞', '動詞', '記号']:
            r.append(w)
    rl = (" ".join(r)).strip()
    results.append(rl)
    print(rl) # 画面に分かち書きした行を表示

上 　 先生 私


一

私 人 先生 呼ぶ いる 。 ここ 先生 書く 本名 打ち明ける 。 これ 世間 憚 かる 遠慮 いう 、 方 私 自然 。 私 人 記憶 呼ぶ 起す ごと 、 「 先生 」 いう なる 。 筆 執る 心持 事 。 よそよそしい 頭文字 使う 気 なる 。
私 先生 知り合い なる の 鎌倉 。 時 私 若々しい 書生 。 暑中 休暇 利用 する 海水浴 行く 友達 来る 端書 受け取る 、 私 金 工面 する 、 出掛ける 事 する 。 私 金 工面 二 、 三 日 費やす 。 私 鎌倉 着く 三 日 経つ うち 、 私 呼び寄せる 友達 、 急 国元 帰る 電報 受け取る 。 電報 母 病気 断る ある 友達 それ 信じる 。 友達 かねる 国元 いる 親 たち 勧 ない 結婚 強いる られる いる 。 彼 現代 習慣 いう 結婚 する 年 若い 過ぎる 。 それ 肝心 当人 気に入る 。 夏休み 帰る ところ 、 避ける 東京 近く 遊ぶ いる の 。 彼 電報 私 見せる する 相談 する 。 私 いい 分る 。 彼 母 病気 する 彼 固い 帰る はず 。 彼 帰る 事 なる 。 来る 私 一 人 取り残す れる 。
学校 授業 始まる 大分 日数 ある 鎌倉 おる よい 、 帰る よい 境遇 いる 私 、 当分 元 宿 留まる 覚悟 する 。 友達 中国 ある 資産 家 息子 金 不自由 ない 男 、 学校 学校 の 年 年 、 生活 程度 私 変り する 。 一人ぼっち なる 私 恰好 宿 探す 面倒 もつ の 。
宿 鎌倉 辺鄙 方角 ある 。 玉突き アイスクリーム の ハイカラ もの 長い 畷 一つ 越す 手 届く 。 車 行う 二 十 銭 取る れる 。 個人 別荘 そこ ここ いくつ 建てる られる いる 。 それ 海 近い 海水浴 やる 便利 地位 占める いる 。
私 毎日 海 はいる 出掛ける 。 古い 燻 ぶり 返る 藁葺 間 通り抜ける 磯 下りる 、 辺 これ 都会 人種 住む いる 思う 、 避暑 来る 男 女 砂 上 動く いる 。 時 海 中 銭湯 よう 黒い 頭 する いる 事 ある 。 中 知る 人 一 人 もつ 私 、 賑やか 景色 中 裹 まれ 、 砂 上 寝そべる みる 、 膝頭 波 打

これ 奥さん 特色 ない いう 、 特色 示す 機会 来る の 解釈 する 方 正当 知れる 。 私 いつ 先生 付属 する 一部分 よう 心持 奥さん 対す いる 。 奥さん 自分 夫 所 来る 書生 好意 、 私 遇 する いる 。 中間 立つ 先生 取り除ける 、 つまり 二 人 ばらばら なる いる 。 知り合い なる 時 奥さん 、 美しい 外 何 感じ 残る いる 。
時 私 先生 宅 酒 飲ます れる 。 時 奥さん 出る 来る 傍 酌 する くれる 。 先生 愉快 そう 見える 。 奥さん 「 お前 一つ 上がり 」 いう 、 自分 呑む 干す 盃 差す 。 奥さん 「 私 … … 」 辞退 しかける 後 、 迷惑 そう それ 受け取る 。 奥さん 綺麗 眉 寄せる 、 私 半分 注ぐ 上げる 盃 、 唇 先 持つ 行く 。 奥さん 先生 間 下 よう 会話 始まる 。
「 珍 事 。 私 呑む おっしゃる 事 滅多 」
「 お前 嫌い 。 稀 飲む いい 。 好い 心持 なる 」
「 なる 。 苦しい ぎりで 。 あなた 大変 愉快 、 酒 召し上がる 」
「 よる 大変 愉快 なる 。 いつ わけ いく 」
「 今夜 いかが 」
「 今夜 好い 心持 」
「 毎晩 召し上がる ざんする 」
「 いく 」
「 召し上がる 下さる 。 方 淋しい 好い 」
先生 宅 夫婦 下女 。 行く たび 大抵 ひす する いる 。 高い 笑い声 聞こえる 試し ない 。 時 宅 中 いる もの 先生 私 よう 気 する 。
「 子供 ある 好い ん 」 奥さん 私 方 向く いく 。 私 「 」 答える 。 私 心 何 同情 起る 。 子供 持つ 事 ない 時 私 、 子供 蒼 蠅 いも よう 考える いる 。
「 一 人 ｜ 貰う やる 」 先生 いう 。
「 貰 ッ 子 、 あなた 」 奥さん 私 方 向く 。
「 子供 いつ 経つ きる くる 」 先生 いう 。
奥さん 黙る いる 。 「 」 私 代り 聞く 時 先生 「 天罰 」 いう 高い 笑う 。

九

私 知る 限り 先生 奥さん 、 仲 好い 夫婦 一 対 。 家庭 一員 暮す 事 ない 私 こと 、 深い 消息 ｜ 解る 、 座敷 私 対坐 する いる 時 、 先生 何 ついで 、 下女 

「 滅多 出る 事 ある 。 近頃 人 顔 見る の 嫌い なる よう 」
いう 奥さん 様子 、 困る もの 風 見える 、 私 大胆 なる 。
「 それ 奥さん 例外 ん 」
「 私 嫌う れる いる 一 人 ん 」
「 嘘 」 私 いう 。 「 奥さん 自身 嘘 知る おっしゃる ん 」
「 」
「 私 いう せる 、 奥さん 好き なる 世間 嫌い なる ん もの 」
「 あなた 学問 する 方 ある 、 上手 。 空っぽ 理屈 使いこなす 事 。 世の中 嫌い なる 、 私 嫌い なる ん いう れる ある 。 なじる 理屈 」
「 両方 いう れる 事 いう れる 、 場合 私 方 正しい の 」
「 議論 いや 。 男 方 議論 なさる の 、 面白い そう 。 空 盃 飽きる 献酬 できる 思う 」
奥さん 言葉 手痛い 。 言葉 耳 障る いう 、 猛烈 もの ない 。 自分 頭脳 ある 事 相手 認める させる 、 そこ 一種 誇り 見出す 奥さん 現代 的 。 奥さん それ 底 方 沈む 心 大事 する いる 見える 。

十 七

私 その後 いう 事 もつ いる 。 奥さん 徒 ら 議論 仕掛ける 男 よう 取る れる 困る 思う 遠慮 する 。 奥さん 飲み干す 紅茶 茶碗 底 覗く 黙る いる 私 外らす よう 、 「 一 杯 上げる 」 聞く 。 私 茶碗 奥さん 手 渡す 。
「 いくつ ？ 　 一つ ？ 　 二 ッ ？ 」
妙 もの 角砂糖 つまみ上げる 奥さん 、 私 顔 見る 、 茶碗 中 入れる 砂糖 数 聞く 。 奥さん 態度 私 媚びる いう ない 、 先刻 強い 言葉 力む 打ち消す する 愛嬌 充ちる いる 。
私 黙る 茶 飲む 。 飲む しまう 黙る いる 。
「 あなた 大変 黙り込む じまう 」 奥さん いう 。
「 何 いう 議論 仕掛ける 、 叱り付ける られる そう 」 私 答える 。
「 」 奥さん いう 。
二 人 それ 緒 口 話 始める 。 二 人 共通 興味 ある 先生 問題 する 。
「 奥さん 、 先刻 続き いう せる 下さる 。 奥さん 空 理屈 聞こえる 知れる 、 私 上の空 いう てる 事 ん 」
「 おっしゃる 」
「 奥さん 急 いる なる する 、 先生 現在 通り 生きる

父 勝つ 時 一番 やる いう 。 くせ 負ける 時 、 一番 やる いう 。 、 勝つ 負ける 、 炬燵 、 碁 差す がる 男 。 始め うち 珍しい 、 隠居 じみる 娯楽 私 相当 興味 与える 、 時日 経つ 伴 れる 、 若い 私 気力 くらい 刺戟 満足 できる なる 。 私 金 香車 握る 拳 頭 上 伸ばす 、 思い切る あくび する 。
私 東京 事 考える 。 する 漲る 心臓 血潮 奥 、 活動 活動 打つ つづける 鼓動 聞く 。 不思議 鼓動 音 、 微妙 意識 状態 、 先生 力 強める られる いる よう 感じる 。
私 心 うち 、 父 先生 比較 する 見る 。 両方 世間 見る 、 生きる いる 死ぬ いる 分る 大人しい 男 。 他 認める られる 点 いう どっち 零 。 、 将 碁 差す がる 父 、 娯楽 相手 私 物足りない 。 遊興 ため 往来 する 覚え ない 先生 、 歓楽 交際 出る 親しみ 以上 、 いつか 私 頭 影響 与える いる 。 頭 の 冷やか 過ぎる 、 私 胸 いう 直す 。 肉 なか 先生 力 喰う 込む いる いう 、 血 なか 先生 命 流れる いる いう 、 時 私 誇張 よう 思う れる 。 私 父 私 本当 父 、 先生 いう ない 、 あか 他人 明白 事実 、 眼 前 並べる みる 、 真理 発見 する とく 驚く 。
私 そつ する 出す 前後 する 、 父 母 眼 今 珍しい 私 ｜ 陳腐 なる 来る 。 これ 夏休み 国 帰る 誰 一様 経験 する 心持 思う 、 当座 一 週間 下 置く よう 、 歓待 する れる 、 峠 定規 通り 通り越す 、 あと 家族 熱 冷める 来る 、 有る 無い 構う もの よう 粗末 取り扱う れる がち なる もの 。 私 滞在 中 峠 通り越す 。 上 私 国 帰る たび 、 父 母 解る 変 ところ 東京 持つ 帰る 。 昔 いう 、 儒者 家 切支丹 臭い 持ち込む よう 、 私 持つ 帰る もの 父 母 調和 する 。 私 それ 隠す いる 。 身 着く いる もの 、 出す 思う 、 いつか それ 父 母 眼 留まる 。 私 面白い なる 。 早い 東京 帰る なる 。
父 病気 現状 維持 まま 、 悪い 方 進む 模様 見

私 先生 返事 平凡 過ぎる 詰る 。 先生 調子 乗る 、 私 拍子抜け 気味 。 私 澄ます 歩く 出す 。 先生 後れ がち なる 。 先生 あと 「 」 声 掛ける 。
「 見る まう 」
「 何 」
「 君 気分 、 私 返事 一つ 変る 」
待ち合わせる ため 振り向く 立つ 留まる 私 顔 見る 、 先生 いう 。

三 十

時 私 腹の中 先生 憎らしい 思う 。 肩 並べる 歩く 出す 、 自分 聞く 事 聞く いる 。 先生 方 、 それ 気が付く いる の 、 いる の 、 私 態度 拘泥 様子 見せる 。 通り 沈黙 がち 落ちる 付き 払う 歩調 すます 運ぶ 行く 、 私 業腹 なる 。 いく 一つ 先生 やる 付ける みる なる 来る 。
「 先生 」
「 何 」
「 先生 さっき 昂奮 なさる 。 植木 屋 庭 休む いる 時 。 私 先生 昂奮 する の 滅多 見る 事 ない ん 、 今日 珍しい ところ 拝見 する よう 気 する 」
先生 返事 する 。 私 それ 手応え ある よう 思う 。 的 外れる よう 感じる 。 仕方 ない 後 いう 事 する 。 先生 道 端 寄る 行く 。 綺麗 刈り込む 生垣 下 、 裾 まくる 小便 する 。 私 先生 用 足す 間 そこ 立つ いる 。
「 やあ 失敬 」
先生 いう 歩く 出す 。 私 先生 やり込める 事 断念 する 。 私 たち 通る 道 ｜ 賑やか なる 。 今 見える 広い 畠 斜面 平地 、 眼 入る よう 左右 家並 揃う くる 。 所々 宅地 隅 、 豌豆 蔓 竹 からませる 、 金網 鶏 囲う 飼う する する の 閑静 眺める られる 。 市中 帰る 駄馬 仕切り ない 擦れ違う 行く 。 もの 気 られる がち 私 、 さっき 胸 中 ある 問題 どこ 振り 落す しまう 。 先生 そこ 後戻り する 時 、 私 それ 忘れる いる 。
「 私 先刻 昂奮 する よう 見える ん 」
「 いう ある 、 … … 」
「 見える 構う 。 ｜ 昂奮 する ん 。 私 財産 事 いう 昂奮 する ん 。 君 見える 知る 、 私 これ 大変 執念深い 男 ん 。 人 受ける 屈辱 損害 、 十 年 たつ 二 十 年 たつ 忘れる ん 」
先生 言葉 元 昂

...

今 書物 城壁 きずく 中 立てる 籠る いる よう Ｋ 心 、 打ち解ける 来る の 見る いる の 、 私 取る 何 愉快 。 私 最初 目的 事 やる 出す の 、 自分 成功 伴う 喜悦 感ずる いる られる の 。 私 本人 いう 代り 、 奥さん お嬢さん 自分 思う 通り 話す 。 二 人 満足 様子 。

二 十 六

「 Ｋ 私 科 おりる 、 専攻 学問 違う いる 、 自然 出る 時 帰る 時 遅速 ある 。 私 方 早い 、 彼 空 室 通り抜ける 、 遅い 簡単 挨拶 する 自分 部屋 はいる の 例 する いる 。 Ｋ 眼 書物 はなす 、 襖 開ける 私 見る 。 今 帰る の いう 。 私 何 答える 点頭く 事 ある 、 ただ 「 」 答える 行く 過ぎる 場合 ある 。
日 私 神田 用 ある 、 帰り 後れる 。 私 急ぎ足 門前 来る 、 格子 開ける 。 それ 、 私 お嬢さん 声 聞く の 。 声 慥か Ｋ 室 出る 思う 。 玄関 真直 行く 、 茶の間 、 お嬢さん 部屋 二つ 続く いる 、 それ 左 折れる 、 Ｋ 室 、 私 室 、 間 取る の 、 どこ 誰 声 する 、 久しい 厄介 なる いる 私 分る の 。 私 格子 締める 。 お嬢さん 声 已 みる 。 私 靴 脱ぐ いる うち 、 ―― 私 時分 ハイカラ 手数 かかる 編 上 穿く いる の 、 ―― 私 こごむ 靴 紐 解く いる うち 、 Ｋ 部屋 誰 声 する 。 私 変 思う 。 よる 、 私 疳 違 知れる 考える の 。 私 通り Ｋ 室 抜ける 、 襖 開ける 、 そこ 二 人 坐る いる 。 Ｋ 例 通り 今 帰る いう 。 お嬢さん 「 帰り 」 坐る まま 挨拶 する 。 私 気 せい 簡単 挨拶 硬い よう 聞こえる 。 どこ 自然 踏み外す いる よう 調子 、 私 鼓膜 響く の 。 私 お嬢さん 、 奥さん 尋ねる 。 私 質問 何 意味 ある 。 家 うち 平常 する いる 聞く 見る 事 。
奥さん 留守 。 下女 奥さん いっしょ 出る の 。 家 残る いる の 、 Ｋ お嬢さん の 。 私 首 傾ける 。 今 長い 間 世話 なる いる 、 奥さん お嬢さん 私 置き去り する 、 宅 空ける 例 ない の 

私 そこ 坐る まま 書見 する 。 宅 中 静まる 、 誰 話し声 聞こえる うち 、 初冬 寒い さ 佗 びしさとが 、 私 身体 食い込む よう 感じ する 。 私 書物 伏せる 立ち上る 。 私 賑やか 所 行く なる の 。 雨 歇 く よう 、 空 冷たい 鉛 よう 重い 見える 、 私 用心 ため 、 蛇の目 肩 担ぐ 、 砲兵 工廠 裏手 土塀 東 坂 下りる 。 時分 道路 改正 できる 頃 、 坂 勾配 今 急 。 道幅 狭い 、 真直 ない の 。 上 谷 下りる 、 南 高い 建物 塞がる いる の 、 放水 よい の 、 往来 。 細い 石橋 渡る 柳 町 通り 出る 間 非道 かう の 。 足駄 長靴 むやみ 歩く 訳 ゆく 。 誰 路 真中 自然 細長い 泥 掻き分ける られる 所 、 後生大事 辿る 行く なる の 。 幅 一 、 二 ｜ 尺 ない の 、 手 ない 往来 敷く ある 帯 上 踏む 向う 越す の 事 。 行く 人 みんな 一 列 なる 通り抜ける 。 私 細い 帯 上 、 Ｋ 出合う 。 足 方 気 取る れる いる 私 、 彼 向き合う 、 彼 存在 気が付く いる の 。 私 不意 自分 前 塞がる 偶然 眼 上げる 時 、 始める そこ 立つ いる Ｋ 認める の 。 私 Ｋ どこ 行く の 聞く 。 Ｋ そこ いう たぎる 。 彼 答え 通り ふん 調子 。 Ｋ 私 細い 帯 上 身体 替る せる 。 Ｋ 後ろ 一 人 若い 女 立つ いる の 見える 。 近眼 私 、 今 それ 分る の 、 Ｋ やる 越す 後 、 女 顔 見る 、 それ 宅 お嬢さん 、 私 少ない 驚く 。 お嬢さん 心持 薄 赤い 顔 する 、 私 挨拶 する 。 時分 束髪 今 違う 廂 出る いる の 、 頭 真中 蛇 よう 巻く つける ある もの 。 私 お嬢さん 頭 見る いる 、 次 瞬間 、 どっち 路 譲る なる の 事 気が付く 。 私 思い切る 中 片足 ｜ 踏む 込む 。 通る やすい 所 空ける 、 お嬢さん 渡す やる 。
柳 町 通り 出る 私 どこ 行く 好い 自分 分る なる 。 どこ 行く 面白い よう 心持 する の 。 私 飛 泥 上がる の 構う 、 糠 海 中 自暴 歩く 。 それ 宅 帰る 

いう しまう 大変 簡単 聞こえる 、 心 経過 、 潮 満干 よう 、 高低 ある の 。 私 Ｋ 動く 様子 見る 、 それ さまざま 意味 付け加える 。 奥さん お嬢さん 言語 動作 観察 する 、 二 人 心 そこ 現われる いる 通り の 疑う みる 。 人間 胸 中 装置 する れる 複雑 器械 、 時計 針 よう 、 明瞭 偽り ない 、 盤上 数字 指す 得る もの 考える 。 私 事 取る 、 取る する 揚句 、 ここ 落ちる 付く もの 思う 下さる 。 むずかしい いう 、 落ちる 付く 言葉 、 際 使う れる 義理 の 知れる 。
内 学校 始まる 。 私 たち 時間 日 連れ立つ 宅 出る 。 都合 よい 帰る 時 いっしょ 帰る 。 外部 見る Ｋ 私 、 何 前 違う ところ ない よう 親しい なる の 。 腹の中 、 各自 各自 事 勝手 考える いる 違い ある 。 日 私 往来 Ｋ 肉薄 する 。 私 一 聞く の 、 この間 自白 私 限る れる いる 、 奥さん お嬢さん 通じる いる 点 ある の 。 私 取る 態度 、 問い 彼 答え 次第 極める なる 、 私 思う の 。 彼 外 人 誰 打ち明ける いる 明言 する 。 私 事情 自分 推察 通り 、 内心 ｜ 嬉しい がる 。 私 Ｋ 私 横着 の 知る いる 。 彼 度胸 敵 ない 自覚 ある の 。 一方 妙 彼 信じる いる 。 学資 事 養家 三 年 欺く いる 彼 、 彼 信用 私 損 われる いる の 。 私 それ ため 彼 信じる 出す 。 疑い深い 私 、 明白 彼 答え 腹の中 否定 する 気 起り よう ない の 。
私 彼 向う 、 彼 恋 取り扱う つもり 尋ねる 。 それ 自白 過ぎる の 、 自白 ついで 、 的 効果 収める 気 の 問う の 。 彼 そこ なる 、 何 答える 。 黙る 下 向く 歩く 出す 。 私 彼 隠し立て する くれる 、 すべて 思う 通り 話す くれる 頼む 。 彼 何 私 隠す 必要 ない 判然 断言 する 。 私 知る する 点 、 一言 返事 与える の 。 私 往来 立つ 留まる 底 突く 留める 訳 いく 。 それなり する しまう 。

四 十

「 日 私 久しぶり 学校 図書館 入る

二つ 三つ 問答 ある 、 私 それ 忘れる しまう 。 男 よう 判然 する ところ ある 奥さん 、 普通 女 違う 場合 大変 心持 よい 話 できる 人 。 「 ざんする 、 差し上げる 」 いう 。 「 差し上げる 威張る 口 利 境遇 ある 。 貰う 下さる 。 ご存じ 通り 父親 ない 憐れ 子 」 後 向う 頼む 。
話 簡単 明瞭 片付く しまう 。 最初 する 十 五 分 掛る 。 奥さん 何 条件 持ち出す の 。 親類 相談 する 必要 ない 、 後 断る 沢山 いう 。 本人 意嚮 たしかめる 及ぶ 明言 する 。 点 なる 、 学問 する 私 方 、 形式 拘泥 する 思う れる の 。 親類 、 当人 話す 承諾 得る の 順序 私 注意 する 時 、 奥さん 「 大丈夫 。 本人 不承知 所 、 私 子 やる はず ある 」 いう 。
自分 室 帰る 私 、 事 訳 ない 進行 する の 考える 、 変 気持 なる 。 大丈夫 の 疑念 、 どこ 頭 底 這う 込む 来る 。 大体 上 、 私 未来 運命 、 これ 定める られる の 観念 私 すべて 新た する 。
私 午 頃 茶の間 出掛ける 行く 、 奥さん 、 今朝 話 お嬢さん 何 時 通じる くれる つもり 尋ねる 。 奥さん 、 自分 承知 する いる 、 いつ 話す 構う よう 事 いう の 。 なる 私 相手 方 男 みる よう 、 私 それ ぎり 引き込む する 。 奥さん 私 引き留める 、 早い 方 希望 、 今日 いう 、 稽古 帰る 来る 、 話す の 。 私 する もらう 方 都合 好い 答える 自分 室 帰る 。 黙る 自分 机 前 坐る 、 二 人 話 遠く 聞く いる 私 想像 する みる 、 落ちる 付く いる られる よう 気 する の 。 私 帽子 被る 表 出る 。 坂の下 お嬢さん 行き合う 。 何 知る お嬢さん 私 見る 驚く の 。 私 帽子 脱 「 帰り 」 尋ねる 、 向う 病気 癒 く の 不思議 そう 聞く の 。 私 「 癒 、 癒 」 答える 、 水道 橋 方 曲る しまう 。

四 十 六

「 私 猿楽 町 神保 町 通り 出る 、 小川 町 方 曲る 。 私 界隈 歩く の 、 古本屋 ひやかす の 目的 、 日 

酒 止める 、 何 する 気 なる 。 仕方 ない 書物 読む 。 読む 読む 、 打つ 遣る 置く 。 私 妻 何 ため 勉強 する の 質問 受ける 。 私 苦笑 する いる 。 腹 底 、 世の中 自分 信愛 する いる 一 人 人間 、 自分 理解 する いる の 思う 、 悲しい の 。 理解 する せる 手段 ある 、 理解 する せる 勇気 出せる の 思う 悲しい の 。 私 寂寞 。 どこ 切り離す れる 世の中 一 人 住む いる よう 気 する 事 ある 。
私 Ｋ 死因 繰り返す 繰り返し 考える の 。 当座 頭 恋 一 字 支配 する れる いる せい ある 、 私 観察 簡単 直線 的 。 Ｋ 失恋 ため 死ぬ もの 極める しまう の 。 段々 落ち 付く 気分 、 現象 向う みる 、 容易い 解決 着く よう 思う れる 来る 。 現実 理想 衝突 、 ―― 不充分 。 私 Ｋ 私 よう 一 人 淋しい 仕方 なくなる 結果 、 急 所 決す の 疑う 出す 。 慄 する の 。 私 Ｋ 歩く 路 、 Ｋ よう 辿る いる の 予覚 、 風 よう 私 胸 横 過る 始める 。

五 十 四

「 内 ｜ 妻 母 病気 なる 。 医者 見せる 癒 ら ない 診断 。 私 力 及ぶ かぎり 懇切 看護 する やる 。 これ 病人 自身 ため ある 、 愛する 妻 ため ある 、 意味 いう 、 人間 ため 。 私 それ 何 する 堪る の 、 何 する 事 できる やむをえる 懐手 する いる 違い ある 。 世間 切り離す れる 私 、 自分 手 出す 、 幾分 善い 事 する 自覚 得る の 時 。 私 罪滅し 名づける なる 、 一種 気分 支配 する れる いる の 。
母 死ぬ 。 私 妻 二 人 ぎりになりました 。 妻 私 向う 、 世の中 頼り する もの 一 人 なくなる いう 。 自分 自身 頼り する 事 できる 私 、 妻 顔 見る 涙ぐむ 。 妻 不幸 女 思う 。 不幸 女 口 出す いう 。 妻 聞く 。 妻 私 意味 解る の 。 私 それ 説明 する やる 事 できる の 。 妻 泣く 。 私 不断 ひねくれる 考え 彼女 観察 する いる ため 、 事 いう よう なる の 恨む 。
母 亡くなる 

In [5]:
#書き込み先テキストを開く
wakati_file = 'kokoro.wakati'
with open(wakati_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))

In [6]:
# Word2Vecでモデルを作成
data = word2vec.LineSentence(wakati_file)
model = word2vec.Word2Vec(data,
            size=200, window=10, hs=1, min_count=2, sg=1)
model.save('kokoro.model')
print('ok')

ok


### 夏目漱石「こころ」のモデルで遊んでみよう

In [1]:
model = word2vec.Word2Vec.load('kokoro.model')

NameError: name 'word2vec' is not defined

In [8]:
model.most_similar(positive=['海'])

<ipython-input-8-7175e2d2b8db>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['海'])


[('波', 0.9600456953048706),
 ('水', 0.958351731300354),
 ('包む', 0.947168231010437),
 ('帯', 0.9448865652084351),
 ('足', 0.9398243427276611),
 ('細い', 0.9355771541595459),
 ('広い', 0.9352393746376038),
 ('垣', 0.9327819347381592),
 ('照らす', 0.9297919273376465),
 ('上る', 0.9297916889190674)]

In [9]:
model.most_similar(positive=['人間'])

<ipython-input-9-3782465c6af7>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['人間'])


[('悪人', 0.9489439725875854),
 ('困る', 0.9110218286514282),
 ('世間', 0.9089219570159912),
 ('信用', 0.9042813181877136),
 ('焦慮', 0.8971787095069885),
 ('幸福', 0.896264910697937),
 ('間際', 0.8950862288475037),
 ('己', 0.8947649002075195),
 ('疑る', 0.8925949931144714),
 ('世の中', 0.8921098113059998)]

## 夏目漱石・太宰治・芥川龍之介の作品をベクトル化しよう

データがないため保留

## Wikipedia日本語の全文データを辞書に使ってみよう

In [1]:
from gensim.models import word2vec
data = word2vec.Text8Corpus("wiki_wakati.txt")
model = word2vec.Word2Vec(data, size=100)
model.save("wiki.model")
print("ok")

ok


### Wikipediaの全文データで遊んでみよう

In [2]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('wiki.model')

In [3]:
# まずはpythonの類義語を調べる
model.most_similar(positive='Python')

<ipython-input-3-5b6139ef1a48>:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive='Python')


[('Perl', 0.9441567659378052),
 ('JavaScript', 0.9204642176628113),
 ('Java', 0.9160013198852539),
 ('Haskell', 0.8976640701293945),
 ('Lua', 0.8966993093490601),
 ('Lisp', 0.8882718086242676),
 ('MATLAB', 0.8810214996337891),
 ('VBScript', 0.8794577121734619),
 ('Emacs', 0.8708335161209106),
 ('Qt', 0.8701063990592957)]

In [4]:
model.most_similar(positive=['カレー'])

<ipython-input-4-b5c541c0d1d8>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['カレー'])


[('ステーキ', 0.8429057598114014),
 ('ハンバーグ', 0.8420965671539307),
 ('ハヤシライス', 0.8327817916870117),
 ('オムライス', 0.8282991051673889),
 ('パスタ', 0.8219684958457947),
 ('焼きそば', 0.8209033608436584),
 ('チャーハン', 0.8176072239875793),
 ('マーボー', 0.8170512318611145),
 ('シーフード', 0.8159886598587036),
 ('カレーライス', 0.8127802610397339)]

In [6]:
model.most_similar(positive=['岡村'])

<ipython-input-6-9def01cf10b9>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['岡村'])


[('矢部', 0.846888542175293),
 ('山下', 0.8240225911140442),
 ('松村', 0.8208272457122803),
 ('山田', 0.8145827651023865),
 ('高橋', 0.8123871088027954),
 ('山崎', 0.8071630001068115),
 ('遠藤', 0.8043172359466553),
 ('田代', 0.8025649785995483),
 ('松岡', 0.7994716763496399),
 ('山本', 0.7987179160118103)]

In [8]:
model.most_similar(positive=['王様','女'], negative=['男'])

<ipython-input-8-7b4ff53477bb>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['王様','女'], negative=['男'])


[('シンデレラ', 0.6593887209892273),
 ('魔女', 0.6376775503158569),
 ('花嫁', 0.6312661170959473),
 ('お姫様', 0.6206669211387634),
 ('麗し', 0.6194496154785156),
 ('お姫さま', 0.6192010641098022),
 ('神様', 0.6144478917121887),
 ('美女と野獣', 0.6095380783081055),
 ('奥さま', 0.5953486561775208),
 ('奥様', 0.5929758548736572)]

In [10]:
model.most_similar(positive=['アイドル','男'], negative=['女'])

<ipython-input-10-002f8969c54a>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['アイドル','男'], negative=['女'])


[('ガールズユニット', 0.7005013823509216),
 ('ガールズバンド', 0.7004978656768799),
 ('YouTuber', 0.6973557472229004),
 ('インディーズバンド', 0.6956052184104919),
 ('芸人', 0.6925316452980042),
 ('KARA', 0.6794131398200989),
 ('VTuber', 0.6535344123840332),
 ('パフォーマー', 0.653451681137085),
 ('WAGE', 0.6459579467773438),
 ('スーパースター', 0.6403546333312988)]

In [12]:
model.most_similar(positive=['結婚'], negative=['愛'])

<ipython-input-12-7835f8e5bd90>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['結婚'], negative=['愛'])


[('再婚', 0.6103498935699463),
 ('離婚', 0.5861755609512329),
 ('死別', 0.5789559483528137),
 ('帰国', 0.559463381767273),
 ('離日', 0.5492461919784546),
 ('襲爵', 0.5434519052505493),
 ('訪日', 0.5249267816543579),
 ('病死', 0.5191144943237305),
 ('急死', 0.5180017948150635),
 ('死去', 0.5148147940635681)]

In [13]:
model.most_similar(positive=['東京', '中国'], negative=['日本'])

<ipython-input-13-ee7589588a3c>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['東京', '中国'], negative=['日本'])


[('上海', 0.7288545966148376),
 ('大連', 0.7182403802871704),
 ('北京', 0.7138594388961792),
 ('成都', 0.6954305171966553),
 ('西安', 0.6949070692062378),
 ('瀋陽', 0.6815401315689087),
 ('平壌', 0.6792205572128296),
 ('杭州', 0.666926383972168),
 ('ハルビン', 0.6612157821655273),
 ('蘇州', 0.6610155701637268)]

In [14]:
model.most_similar(positive=['京都','温泉'], negative=['歴史'])

<ipython-input-14-0b342ce0cf1e>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['京都','温泉'], negative=['歴史'])


[('熱海', 0.5918833017349243),
 ('大阪', 0.5896776914596558),
 ('旅館', 0.5723772048950195),
 ('草津', 0.5585567951202393),
 ('嵐山', 0.5546832084655762),
 ('名張', 0.5417994856834412),
 ('城山観光', 0.5368615388870239),
 ('湯', 0.5316998362541199),
 ('茶屋', 0.5303549766540527),
 ('城崎', 0.5299674868583679)]

In [16]:
model.most_similar(positive=['カレー', '辛い'])

<ipython-input-16-4cc13e05ad68>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['カレー', '辛い'])


[('美味しい', 0.8306519985198975),
 ('カレーライス', 0.8187036514282227),
 ('シチュー', 0.8078387975692749),
 ('不味い', 0.8069307804107666),
 ('オムライス', 0.8034468293190002),
 ('カップラーメン', 0.7891316413879395),
 ('キムチ', 0.7890461683273315),
 ('酸っぱい', 0.7860251665115356),
 ('トンカツ', 0.7856348752975464),
 ('美味い', 0.7839035987854004)]

# ベイズの定理でテキストを分類しよう

## ベイズの定理を紹介

P(B|A) = P(A|B)P(B)P(A)

## ナイーブベイズ分類について

ナイーブベイズ分類器というのはベイズの定理を利用した分類手法。  
ベイズの定理では、Aには何らかの起こった事象を、Bにはそこから推論したい事柄などを当てはめる。そのため、Aを入力テキストとするなら、Bは決定されるカテゴリであるということになる。もし、これがメールのスパムフィルタであれば、Aが受信したメールのテキストで、Bがスパムかどうかの判定結果となる。  
ナイーブベイズ分類では、ある文章をカテゴリ分けする際に、テキスト中の単語の出現率を調べる。その際にその文章をどのカテゴリに分類するのがふさわしいかを調べる。  
実際に判定を行う時、P(B|A)の値は、1つの確率の値ではなく、複数のカテゴリの中で、とのカテゴリになる確率が一番高いかを表す情報となる。このとき、ベイズ定理の分母であるP(A)は入力テキストが与えられる確率となる。しかし、どのカテゴリを判定にするにしても、同じ入力テキストが与えられるため同じ値になると考えて、考慮しないこととすると、ナイーブベイズ分類の式はより単純になる。  
P(B|A) = P(B) * P(A|B)  
この時、P(B)は事前確率であり、各カテゴリに分類される確率を表している。学習したデータの各カテゴリの総文書数に占める割合となる。  
P(A|B)について考えると、入力テキストAは各単語の集合と考えることができる。テキスト内のテキストを単語に分割するということである。この時、集合であるため、並びは意味をなさない。単語が文書内のどこに出てくるかは考慮しない。バッグの中に単語が書かれた積み木を放り込んでおくイメージである。そのため、このようなテキスト表現を「BoW=bag-of-words」と呼ぶ。  
入力テキストAを書く単語aNの集合と考えると、P(A|B)を次の式に直すことができる。  
P(A|B) = P(a_a1|B)P(a_a2|B)P(a_a3|B)...P(a_xN|B)  
P(aN|B)の確率は、分割した単語があるカテゴリに属する確率を求めることになる。つまり、あるカテゴリにおいて、その単語が出現した確率を求める。これは、次の計算で求められる。  
単語の出現率 = 単語の出現回数 / カテゴリの全単語数  
入力テキストを学習させて訓練させるときは、出現した単語が各カテゴリに分類された回数を覚えておけばいいことになる。そして、カテゴリ分の推定を行うときは、P(B)*P(A|B)をカテゴリごとに計算する。


## ベイジアンフィルタを使ってみよう

In [17]:
import math, sys
from janome.tokenizer import Tokenizer # 形態素解析

In [28]:
class BayesianFilter:
    """ベイジアンフィルタ"""
    def __init__(self):
        self.words = set() # 出現した単語をすべて記録
        self.word_dict = {} # カテゴリごとの単語出現回数を記録
        self.category_dict = {} # カテゴリの出現回数を記録
        
    # 形態素解析を行う
    def split(self, text):
        result = []
        t = Tokenizer()
        malist = t.tokenize(text)
        for w in malist:
            sf = w.surface # 区切られた単語そのまま
            bf = w.base_form # 単語の基本系
            if bf=='' or bf=="*":
                bf = sf
            result.append(bf)
        return result
    
    # 単語とカテゴリを数える処理
    def inc_word(self, word, category):
        # 単語をカテゴリに追加
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)
    def inc_category(self, category):
        # カテゴリを加算する
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1
        
    # テキストを学習する
    def fit(self, text, category):
        """テキストの学習"""
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word,category)
        self.inc_category(category)
    
    # カテゴリにおける単語リストのスコアを計算する
    def score(self, words, category):
        score = math.log(self.category_prob(category))
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score
    
    # テキスト内のカテゴリ分けを行う
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize
        words = self.split(text)
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list
    
    # カテゴリ内の単語出現数を得る
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
    
    # カテゴリ / 総カテゴリを計算
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories
    
    # カテゴリ内の単語の出現率を計算
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n/d

In [29]:
bf = BayesianFilter()

In [30]:
# テキストを学習
bf.fit("激安セール - 今日だけ三割引","広告")
bf.fit("クーポンプレゼント&送料無料","広告")
bf.fit("店内改装セール実施中","広告")
bf.fit("美味しくなって再登場","広告")
bf.fit("本日の予定の確認です。","重要")
bf.fit("プロジェクトの深緑確認をお願いします。","重要")
bf.fit("打ち合わせよろしくお願いします。","重要")
bf.fit("会議の議事録です","重要")

In [31]:
# 予測
pre, scorelist = bf.predict("激安、在庫一掃セール、送料無料")
print("結果=", pre)
print(scorelist)

結果= 広告
[('広告', -34.391431598712785), ('重要', -38.535380755078634)]


# MLPでテキスト分類してみよう

多層パーセプトロン(Multi Layer Perceptron)とは入力そうと出力層の間に、それぞれ全結合する隠れ層を挟んだニューラルネットワークのこと

### livedoorニュースコーパスをサンプルとして利用

ニュース記事はディレクトリごとに整理されている。各ディレクトリ以下に、たくさんのテキストファイルが保存されている。そこで、今回は、この9カテゴリ、7367このテキストファイルを読んでそれがどのカテゴリのニュース記事になるかを判定するプログラムを作る。

1. トピックニュース
2. Sports Watch
3. ITライフハック
4. 家電チャンネル
5. MOVIE ENTER
6. 独女通信
7. エスマックス
8. livedoor HOMME
9. Perchy

### テキストデータを固定長ベクトルに変換する方法

テキストをベクトルデータに変換する方法には、いくつかの方法があるが、基本的には単語の一つ一つにIDを振って、そのIDの出現頻度やIDの並び順をベクトル するという方法を利用する。  
今回は、単語の並び順は無視して、単語の出現頻度だけを利用する。前説で説明したように、文章中にどんな単語があるかを数値で表す手法をBoW (Bag-of-words)と呼ぶ。  
例えば、「何度倒れても何度も立ち上がる」という文をBoWで表してみる。まずは形態素解析を行う。  

何度|倒れ|て|も|何度|も|立ち上がる  

そして、各単語にIDを振って、置き換えてみる。  

|形態素|何度|倒れ|て|も|何度|も|立ち上がる|
|:--- |:--- |:--- |:--- |:--- |:--- |:--- |:---|
|ID|1|2|3|4|1|4|5|  

これだけで、文章をベクトル表現にすることができた。そして、このID表現の、単語の並び順の情報を無視して、単語の出現回数を数えてみる。  

|形態素|何度|倒れ|て|も|立ち上がる|
|:--- |:--- |:--- |:--- |:--- |:--- |
|ID|1|2|3|4|5|  
|出現回数|2|1|1|2|1|  

このようにして、単語の出現回数を文章を表すベクトルとするならば、どの文章に対しても固定長のデータが得られる。

### テキストの分類に挑戦してみよう

テキストを分類するまでの手順を確認する。  
1. テキストを分かち書きし、助詞や記号など不要な品詞を捨てる
2. 辞書を作り、単語を数値に変換する
3. ファイル内の単語の出現率を求める
4. データを学習させる
5. テストデータを振り分けて成功率を確認する

### テキストの分かち書き

In [2]:
from janome.tokenizer import Tokenizer
import os, glob

In [3]:
# Janomeを使って形態素解析を行う
ja_tokenizer = Tokenizer()

In [4]:
# 日本語を分かち書き
def ja_tokenize(text):
    res = []
    lines = text.split("\n")
    lines = lines[2:] # 最初の二行はヘッダー情報なので捨てる
    for line in lines:
        malist = ja_tokenizer.tokenize(line)
        for tok in malist:
            ps = tok.part_of_speech.split(",")[0]
            if not ps in ['名詞', '動詞', '形容詞']:
                continue
            w = tok.base_form
            if w == "*" or w == "":
                w = tok.surface
            if w == "" or w == "\n":
                continue
            res.append(w)
        res.append("\n")
    return res

In [5]:
# テストデータを読み込み
root_dir = './newstext'
for path in glob.glob(root_dir+"/*/*.txt", recursive=True):
    if path.find("LICENSE") > 0:
        continue
    print(path)
    path_wakati = path + '.wakati'
    if os.path.exists(path_wakati):
        continue
    text = open(path, "r").read()
    words = ja_tokenize(text)
    wt = " ".join(words)
    open(path_wakati, "w", encoding="utf-8").write(wt)

./newstext/movie-enter/movie-enter-5978741.txt
./newstext/movie-enter/movie-enter-6322901.txt
./newstext/movie-enter/movie-enter-6176324.txt
./newstext/movie-enter/movie-enter-6573929.txt
./newstext/movie-enter/movie-enter-5914880.txt
./newstext/movie-enter/movie-enter-5878782.txt
./newstext/movie-enter/movie-enter-6354868.txt
./newstext/movie-enter/movie-enter-6035751.txt
./newstext/movie-enter/movie-enter-6689395.txt
./newstext/movie-enter/movie-enter-6375454.txt
./newstext/movie-enter/movie-enter-6689342.txt
./newstext/movie-enter/movie-enter-6581639.txt
./newstext/movie-enter/movie-enter-6395403.txt
./newstext/movie-enter/movie-enter-6631156.txt
./newstext/movie-enter/movie-enter-6075201.txt
./newstext/movie-enter/movie-enter-6408753.txt
./newstext/movie-enter/movie-enter-5972654.txt
./newstext/movie-enter/movie-enter-6671599.txt
./newstext/movie-enter/movie-enter-6512253.txt
./newstext/movie-enter/movie-enter-6331809.txt
./newstext/movie-enter/movie-enter-6070635.txt
./newstext/mo

./newstext/movie-enter/movie-enter-6478266.txt
./newstext/movie-enter/movie-enter-5935297.txt
./newstext/movie-enter/movie-enter-6065956.txt
./newstext/movie-enter/movie-enter-6278767.txt
./newstext/movie-enter/movie-enter-6532347.txt
./newstext/movie-enter/movie-enter-6473694.txt
./newstext/movie-enter/movie-enter-6160423.txt
./newstext/movie-enter/movie-enter-6361791.txt
./newstext/movie-enter/movie-enter-6282378.txt
./newstext/movie-enter/movie-enter-6789964.txt
./newstext/movie-enter/movie-enter-5966397.txt
./newstext/movie-enter/movie-enter-5843659.txt
./newstext/movie-enter/movie-enter-5840350.txt
./newstext/movie-enter/movie-enter-6476075.txt
./newstext/movie-enter/movie-enter-6635692.txt
./newstext/movie-enter/movie-enter-6746669.txt
./newstext/movie-enter/movie-enter-6152335.txt
./newstext/movie-enter/movie-enter-5903240.txt
./newstext/movie-enter/movie-enter-6852205.txt
./newstext/movie-enter/movie-enter-6703309.txt
./newstext/movie-enter/movie-enter-6728832.txt
./newstext/mo

./newstext/movie-enter/movie-enter-6415834.txt
./newstext/movie-enter/movie-enter-6640090.txt
./newstext/movie-enter/movie-enter-6324087.txt
./newstext/movie-enter/movie-enter-6580230.txt
./newstext/movie-enter/movie-enter-6569323.txt
./newstext/movie-enter/movie-enter-6839823.txt
./newstext/movie-enter/movie-enter-6379633.txt
./newstext/movie-enter/movie-enter-5950651.txt
./newstext/movie-enter/movie-enter-6336656.txt
./newstext/movie-enter/movie-enter-6469135.txt
./newstext/movie-enter/movie-enter-6372982.txt
./newstext/movie-enter/movie-enter-6568559.txt
./newstext/movie-enter/movie-enter-6563418.txt
./newstext/movie-enter/movie-enter-6330568.txt
./newstext/movie-enter/movie-enter-6332419.txt
./newstext/movie-enter/movie-enter-6609122.txt
./newstext/movie-enter/movie-enter-6310848.txt
./newstext/movie-enter/movie-enter-6379626.txt
./newstext/movie-enter/movie-enter-6793785.txt
./newstext/movie-enter/movie-enter-6219105.txt
./newstext/movie-enter/movie-enter-6316975.txt
./newstext/mo

./newstext/movie-enter/movie-enter-6130912.txt
./newstext/movie-enter/movie-enter-6852057.txt
./newstext/movie-enter/movie-enter-5909503.txt
./newstext/movie-enter/movie-enter-5949899.txt
./newstext/movie-enter/movie-enter-6674489.txt
./newstext/movie-enter/movie-enter-6545834.txt
./newstext/movie-enter/movie-enter-6444280.txt
./newstext/movie-enter/movie-enter-6662566.txt
./newstext/movie-enter/movie-enter-6836506.txt
./newstext/movie-enter/movie-enter-6511242.txt
./newstext/movie-enter/movie-enter-5876300.txt
./newstext/movie-enter/movie-enter-6429087.txt
./newstext/movie-enter/movie-enter-6317326.txt
./newstext/movie-enter/movie-enter-6278087.txt
./newstext/movie-enter/movie-enter-6636902.txt
./newstext/movie-enter/movie-enter-5883799.txt
./newstext/movie-enter/movie-enter-6316602.txt
./newstext/movie-enter/movie-enter-5916999.txt
./newstext/movie-enter/movie-enter-6155542.txt
./newstext/movie-enter/movie-enter-6571818.txt
./newstext/movie-enter/movie-enter-6636533.txt
./newstext/mo

./newstext/movie-enter/movie-enter-6098034.txt
./newstext/movie-enter/movie-enter-6187666.txt
./newstext/movie-enter/movie-enter-6404940.txt
./newstext/movie-enter/movie-enter-6338574.txt
./newstext/movie-enter/movie-enter-5870671.txt
./newstext/movie-enter/movie-enter-6072412.txt
./newstext/movie-enter/movie-enter-6111189.txt
./newstext/movie-enter/movie-enter-6340197.txt
./newstext/movie-enter/movie-enter-6354684.txt
./newstext/movie-enter/movie-enter-6371410.txt
./newstext/movie-enter/movie-enter-6679522.txt
./newstext/movie-enter/movie-enter-6798904.txt
./newstext/movie-enter/movie-enter-6557895.txt
./newstext/movie-enter/movie-enter-6662128.txt
./newstext/movie-enter/movie-enter-6306671.txt
./newstext/movie-enter/movie-enter-5916938.txt
./newstext/movie-enter/movie-enter-6858134.txt
./newstext/movie-enter/movie-enter-5844065.txt
./newstext/movie-enter/movie-enter-6215816.txt
./newstext/movie-enter/movie-enter-5924578.txt
./newstext/movie-enter/movie-enter-6672663.txt
./newstext/mo

...

./newstext/dokujo-tsushin/dokujo-tsushin-6812547.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6079599.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5215801.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6684115.txt
./newstext/dokujo-tsushin/dokujo-tsushin-4948019.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6332078.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5453372.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5733358.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6313075.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5446569.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6501400.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6148585.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5005857.txt
./newstext/dokujo-tsushin/dokujo-tsushin-4834677.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6758308.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5521492.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5132619.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5212559.txt
./newstext/dokujo-tsushin/dokujo-tsushin-53853

./newstext/dokujo-tsushin/dokujo-tsushin-6617085.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5174573.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5619249.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6036589.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6097420.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5949714.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5578197.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6863452.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6723862.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5239637.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5718368.txt
./newstext/dokujo-tsushin/dokujo-tsushin-4791665.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5641553.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5916990.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5468186.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5311144.txt
./newstext/dokujo-tsushin/dokujo-tsushin-5851239.txt
./newstext/dokujo-tsushin/dokujo-tsushin-6785653.txt
./newstext/dokujo-tsushin/dokujo-tsushin-58786

./newstext/smax/smax-6599204.txt
./newstext/smax/smax-6764325.txt
./newstext/smax/smax-6665985.txt
./newstext/smax/smax-6648624.txt
./newstext/smax/smax-6714173.txt
./newstext/smax/smax-6609650.txt
./newstext/smax/smax-6638274.txt
./newstext/smax/smax-6770542.txt
./newstext/smax/smax-6535918.txt
./newstext/smax/smax-6792538.txt
./newstext/smax/smax-6773922.txt
./newstext/smax/smax-6559342.txt
./newstext/smax/smax-6726248.txt
./newstext/smax/smax-6838453.txt
./newstext/smax/smax-6507564.txt
./newstext/smax/smax-6517950.txt
./newstext/smax/smax-6687337.txt
./newstext/smax/smax-6684602.txt
./newstext/smax/smax-6709532.txt
./newstext/smax/smax-6575124.txt
./newstext/smax/smax-6536009.txt
./newstext/smax/smax-6896144.txt
./newstext/smax/smax-6884226.txt
./newstext/smax/smax-6749881.txt
./newstext/smax/smax-6774533.txt
./newstext/smax/smax-6775177.txt
./newstext/smax/smax-6867630.txt
./newstext/smax/smax-6564968.txt
./newstext/smax/smax-6880915.txt
./newstext/smax/smax-6722991.txt
./newstext

./newstext/smax/smax-6537176.txt
./newstext/smax/smax-6613050.txt
./newstext/smax/smax-6605183.txt
./newstext/smax/smax-6780189.txt
./newstext/smax/smax-6838627.txt
./newstext/smax/smax-6737910.txt
./newstext/smax/smax-6896251.txt
./newstext/smax/smax-6635861.txt
./newstext/smax/smax-6727281.txt
./newstext/smax/smax-6635685.txt
./newstext/smax/smax-6671004.txt
./newstext/smax/smax-6903538.txt
./newstext/smax/smax-6682371.txt
./newstext/smax/smax-6816684.txt
./newstext/smax/smax-6912390.txt
./newstext/smax/smax-6742182.txt
./newstext/smax/smax-6631150.txt
./newstext/smax/smax-6705593.txt
./newstext/smax/smax-6511548.txt
./newstext/smax/smax-6770126.txt
./newstext/smax/smax-6632473.txt
./newstext/smax/smax-6588802.txt
./newstext/smax/smax-6564114.txt
./newstext/smax/smax-6688892.txt
./newstext/smax/smax-6827498.txt
./newstext/smax/smax-6904255.txt
./newstext/smax/smax-6567804.txt
./newstext/smax/smax-6883851.txt
./newstext/smax/smax-6736403.txt
./newstext/smax/smax-6753264.txt
./newstext

./newstext/smax/smax-6600999.txt
./newstext/smax/smax-6890777.txt
./newstext/smax/smax-6730319.txt
./newstext/smax/smax-6536934.txt
./newstext/smax/smax-6761618.txt
./newstext/smax/smax-6683845.txt
./newstext/smax/smax-6713502.txt
./newstext/smax/smax-6753126.txt
./newstext/smax/smax-6693470.txt
./newstext/smax/smax-6722720.txt
./newstext/smax/smax-6756923.txt
./newstext/smax/smax-6512115.txt
./newstext/smax/smax-6749426.txt
./newstext/smax/smax-6648738.txt
./newstext/smax/smax-6890004.txt
./newstext/smax/smax-6896449.txt
./newstext/smax/smax-6917816.txt
./newstext/smax/smax-6834765.txt
./newstext/smax/smax-6778610.txt
./newstext/smax/smax-6630907.txt
./newstext/smax/smax-6569160.txt
./newstext/smax/smax-6845003.txt
./newstext/smax/smax-6872644.txt
./newstext/smax/smax-6762700.txt
./newstext/smax/smax-6770304.txt
./newstext/smax/smax-6721163.txt
./newstext/smax/smax-6604644.txt
./newstext/smax/smax-6635499.txt
./newstext/smax/smax-6695613.txt
./newstext/smax/smax-6538139.txt
./newstext

### 単語をIDに変換し出現回数を数える

In [6]:
import os, glob, json

In [7]:
root_dir = "./newstext"
dic_file = root_dir + "/word-dic.json"
data_file = root_dir + "/data.json"
data_file_min = root_dir + "/data-mini.json"

In [25]:
# 語句を区切ってIDに変換する
word_dic = {"_MAX": 0}
def text_to_ids(text):
    text = text.strip()
    words = text.split(" ")
    result = []
    for n in words:
        n = n.strip()
        if n == "":
            continue
        if not n in word_dic:
            wid = word_dic[n] = word_dic["_MAX"]
            word_dic["_MAX"] += 1
            print(wid, n)
        else:
            wid = word_dic[n]
        result.append(wid)
    return result

In [26]:
# ファイルを読んで固定長シーケンスを返す
def file_to_ids(fname):
    with open(fname, "r") as f:
        text = f.read()
        return text_to_ids(text)

In [27]:
# 辞書に全部の単語を登録する
def register_dic():
    files = glob.glob(root_dir+"/*/*.wakati", recursive=True)
    for i in files:
        file_to_ids(i)

In [28]:
# ファイル内の単語を数える
def count_file_freq(fname):
    cnt = [0 for n in range(word_dic["_MAX"])]
    with open(fname, "r") as f:
        text = f.read().strip()
        ids = text_to_ids(text)
        for wid in ids:
            cnt[wid] += 1
    return cnt

In [45]:
# ジャンルごとにファイルを読み込む
def count_freq(limit = 0):
    X = []
    Y = []
    max_words = word_dic["_MAX"]
    cat_names = []
    for cat in os.listdir(root_dir):
        cat_dir = root_dir + "/" + cat
        if not os.path.isdir(cat_dir):
            continue
        cat_idx = len(cat_names)
        cat_names.append(cat)
        files = glob.glob(cat_dir+"/*.wakati")
        i = 0
        for path in files:
            print(path)
            cnt = count_file_freq(path)
            X.append(cnt)
            Y.append(cat_idx)
            if limit > 0:
                if i > limit:
                    break
                i += 1
    return X, Y

In [46]:
# 単語辞書の作成
if os.path.exists(dic_file):
    word_dic = json.load(open(dic_file))
else:
    register_dic()
    json.dump(word_dic, open(dic_file,"w"))

# ファイルごとの単語出現頻度のベクトルを作る
# テスト用に小規模のデータを用意
X, Y = count_freq(20)
json.dump({"X": X, "Y": Y}, open(data_file_min, "w"))

# 全ファイルを対象にデータを作成
X, Y = count_freq()
json.dump({"X": X, "Y": Y}, open(data_file, "w"))
print("ok")

./newstext/movie-enter/movie-enter-6239924.txt.wakati
./newstext/movie-enter/movie-enter-6568559.txt.wakati
./newstext/movie-enter/movie-enter-6305978.txt.wakati
./newstext/movie-enter/movie-enter-6164401.txt.wakati
./newstext/movie-enter/movie-enter-6512253.txt.wakati
./newstext/movie-enter/movie-enter-6185688.txt.wakati
./newstext/movie-enter/movie-enter-6662932.txt.wakati
./newstext/movie-enter/movie-enter-6221124.txt.wakati
./newstext/movie-enter/movie-enter-6330576.txt.wakati
./newstext/movie-enter/movie-enter-6393435.txt.wakati
./newstext/movie-enter/movie-enter-6120264.txt.wakati
./newstext/movie-enter/movie-enter-6727351.txt.wakati
./newstext/movie-enter/movie-enter-5914722.txt.wakati
./newstext/movie-enter/movie-enter-6040192.txt.wakati
./newstext/movie-enter/movie-enter-6269274.txt.wakati
./newstext/movie-enter/movie-enter-5950519.txt.wakati
./newstext/movie-enter/movie-enter-5990297.txt.wakati
./newstext/movie-enter/movie-enter-6128471.txt.wakati
./newstext/movie-enter/movie

./newstext/smax/smax-6792311.txt.wakati
./newstext/smax/smax-6738444.txt.wakati
./newstext/smax/smax-6896381.txt.wakati
./newstext/smax/smax-6537457.txt.wakati
./newstext/movie-enter/movie-enter-6239924.txt.wakati
./newstext/movie-enter/movie-enter-6568559.txt.wakati
./newstext/movie-enter/movie-enter-6305978.txt.wakati
./newstext/movie-enter/movie-enter-6164401.txt.wakati
./newstext/movie-enter/movie-enter-6512253.txt.wakati
./newstext/movie-enter/movie-enter-6185688.txt.wakati
./newstext/movie-enter/movie-enter-6662932.txt.wakati
./newstext/movie-enter/movie-enter-6221124.txt.wakati
./newstext/movie-enter/movie-enter-6330576.txt.wakati
./newstext/movie-enter/movie-enter-6393435.txt.wakati
./newstext/movie-enter/movie-enter-6120264.txt.wakati
./newstext/movie-enter/movie-enter-6727351.txt.wakati
./newstext/movie-enter/movie-enter-5914722.txt.wakati
./newstext/movie-enter/movie-enter-6040192.txt.wakati
./newstext/movie-enter/movie-enter-6269274.txt.wakati
./newstext/movie-enter/movie-e

./newstext/movie-enter/movie-enter-6463004.txt.wakati
./newstext/movie-enter/movie-enter-5979444.txt.wakati
./newstext/movie-enter/movie-enter-5943414.txt.wakati
./newstext/movie-enter/movie-enter-6458309.txt.wakati
./newstext/movie-enter/movie-enter-6013901.txt.wakati
./newstext/movie-enter/movie-enter-6264047.txt.wakati
./newstext/movie-enter/movie-enter-6283175.txt.wakati
./newstext/movie-enter/movie-enter-6845148.txt.wakati
./newstext/movie-enter/movie-enter-6672663.txt.wakati
./newstext/movie-enter/movie-enter-5860161.txt.wakati
./newstext/movie-enter/movie-enter-5840524.txt.wakati
./newstext/movie-enter/movie-enter-6458806.txt.wakati
./newstext/movie-enter/movie-enter-6508806.txt.wakati
./newstext/movie-enter/movie-enter-5988596.txt.wakati
./newstext/movie-enter/movie-enter-6491179.txt.wakati
./newstext/movie-enter/movie-enter-6689342.txt.wakati
./newstext/movie-enter/movie-enter-6464677.txt.wakati
./newstext/movie-enter/movie-enter-6789964.txt.wakati
./newstext/movie-enter/movie

./newstext/movie-enter/movie-enter-6524381.txt.wakati
./newstext/movie-enter/movie-enter-5899362.txt.wakati
./newstext/movie-enter/movie-enter-6569639.txt.wakati
./newstext/movie-enter/movie-enter-5865707.txt.wakati
./newstext/movie-enter/movie-enter-6149578.txt.wakati
./newstext/movie-enter/movie-enter-6808742.txt.wakati
./newstext/movie-enter/movie-enter-6524541.txt.wakati
./newstext/movie-enter/movie-enter-6703309.txt.wakati
./newstext/movie-enter/movie-enter-6165056.txt.wakati
./newstext/movie-enter/movie-enter-6728832.txt.wakati
./newstext/movie-enter/movie-enter-6354216.txt.wakati
./newstext/movie-enter/movie-enter-6622091.txt.wakati
./newstext/movie-enter/movie-enter-6307444.txt.wakati
./newstext/movie-enter/movie-enter-5975520.txt.wakati
./newstext/movie-enter/movie-enter-6625136.txt.wakati
./newstext/movie-enter/movie-enter-6892597.txt.wakati
./newstext/movie-enter/movie-enter-6595602.txt.wakati
./newstext/movie-enter/movie-enter-6488996.txt.wakati
./newstext/movie-enter/movie

./newstext/movie-enter/movie-enter-5952392.txt.wakati
./newstext/movie-enter/movie-enter-6253008.txt.wakati
./newstext/movie-enter/movie-enter-6431636.txt.wakati
./newstext/movie-enter/movie-enter-6361857.txt.wakati
./newstext/movie-enter/movie-enter-6590065.txt.wakati
./newstext/movie-enter/movie-enter-6724187.txt.wakati
./newstext/movie-enter/movie-enter-6743549.txt.wakati
./newstext/movie-enter/movie-enter-6013876.txt.wakati
./newstext/movie-enter/movie-enter-6191062.txt.wakati
./newstext/movie-enter/movie-enter-6900874.txt.wakati
./newstext/movie-enter/movie-enter-6485134.txt.wakati
./newstext/movie-enter/movie-enter-6542182.txt.wakati
./newstext/movie-enter/movie-enter-5910270.txt.wakati
./newstext/movie-enter/movie-enter-6438775.txt.wakati
./newstext/movie-enter/movie-enter-6119832.txt.wakati
./newstext/movie-enter/movie-enter-6050625.txt.wakati
./newstext/movie-enter/movie-enter-6383596.txt.wakati
./newstext/movie-enter/movie-enter-6261452.txt.wakati
./newstext/movie-enter/movie

...

./newstext/smax/smax-6713067.txt.wakati
./newstext/smax/smax-6838514.txt.wakati
./newstext/smax/smax-6712883.txt.wakati
./newstext/smax/smax-6633904.txt.wakati
./newstext/smax/smax-6904760.txt.wakati
./newstext/smax/smax-6896449.txt.wakati
./newstext/smax/smax-6611601.txt.wakati
./newstext/smax/smax-6671368.txt.wakati
./newstext/smax/smax-6669158.txt.wakati
./newstext/smax/smax-6893827.txt.wakati
./newstext/smax/smax-6621020.txt.wakati
./newstext/smax/smax-6719993.txt.wakati
./newstext/smax/smax-6540502.txt.wakati
./newstext/smax/smax-6599022.txt.wakati
./newstext/smax/smax-6730204.txt.wakati
./newstext/smax/smax-6693792.txt.wakati
./newstext/smax/smax-6792311.txt.wakati
./newstext/smax/smax-6738444.txt.wakati
./newstext/smax/smax-6896381.txt.wakati
./newstext/smax/smax-6537457.txt.wakati
./newstext/smax/smax-6815664.txt.wakati
./newstext/smax/smax-6767827.txt.wakati
./newstext/smax/smax-6589045.txt.wakati
./newstext/smax/smax-6665785.txt.wakati
./newstext/smax/smax-6898486.txt.wakati


./newstext/smax/smax-6589268.txt.wakati
./newstext/smax/smax-6894795.txt.wakati
./newstext/smax/smax-6561866.txt.wakati
./newstext/smax/smax-6637825.txt.wakati
./newstext/smax/smax-6789801.txt.wakati
./newstext/smax/smax-6654779.txt.wakati
./newstext/smax/smax-6792799.txt.wakati
./newstext/smax/smax-6761238.txt.wakati
./newstext/smax/smax-6592679.txt.wakati
./newstext/smax/smax-6671169.txt.wakati
./newstext/smax/smax-6828955.txt.wakati
./newstext/smax/smax-6615161.txt.wakati
./newstext/smax/smax-6912200.txt.wakati
./newstext/smax/smax-6611523.txt.wakati
./newstext/smax/smax-6850114.txt.wakati
./newstext/smax/smax-6768164.txt.wakati
./newstext/smax/smax-6863613.txt.wakati
./newstext/smax/smax-6683845.txt.wakati
./newstext/smax/smax-6631010.txt.wakati
./newstext/smax/smax-6725617.txt.wakati
./newstext/smax/smax-6604901.txt.wakati
./newstext/smax/smax-6901408.txt.wakati
./newstext/smax/smax-6741096.txt.wakati
./newstext/smax/smax-6771357.txt.wakati
./newstext/smax/smax-6623092.txt.wakati


./newstext/smax/smax-6917854.txt.wakati
./newstext/smax/smax-6619869.txt.wakati
./newstext/smax/smax-6902669.txt.wakati
./newstext/smax/smax-6912390.txt.wakati
./newstext/smax/smax-6792560.txt.wakati
./newstext/smax/smax-6732193.txt.wakati
./newstext/smax/smax-6792898.txt.wakati
./newstext/smax/smax-6564609.txt.wakati
./newstext/smax/smax-6560368.txt.wakati
./newstext/smax/smax-6575124.txt.wakati
./newstext/smax/smax-6759825.txt.wakati
./newstext/smax/smax-6842219.txt.wakati
./newstext/smax/smax-6750933.txt.wakati
./newstext/smax/smax-6557974.txt.wakati
./newstext/smax/smax-6562624.txt.wakati
./newstext/smax/smax-6731696.txt.wakati
./newstext/smax/smax-6753758.txt.wakati
./newstext/smax/smax-6898846.txt.wakati
./newstext/smax/smax-6635861.txt.wakati
./newstext/smax/smax-6594409.txt.wakati
./newstext/smax/smax-6573652.txt.wakati
./newstext/smax/smax-6808141.txt.wakati
./newstext/smax/smax-6537176.txt.wakati
./newstext/smax/smax-6890004.txt.wakati
./newstext/smax/smax-6817640.txt.wakati


./newstext/smax/smax-6632638.txt.wakati
./newstext/smax/smax-6792463.txt.wakati
./newstext/smax/smax-6902275.txt.wakati
./newstext/smax/smax-6879979.txt.wakati
./newstext/smax/smax-6556511.txt.wakati
./newstext/smax/smax-6821176.txt.wakati
./newstext/smax/smax-6568892.txt.wakati
./newstext/smax/smax-6798184.txt.wakati
./newstext/smax/smax-6705593.txt.wakati
./newstext/smax/smax-6910027.txt.wakati
./newstext/smax/smax-6723772.txt.wakati
./newstext/smax/smax-6869457.txt.wakati
./newstext/smax/smax-6659937.txt.wakati
./newstext/smax/smax-6604645.txt.wakati
./newstext/smax/smax-6552663.txt.wakati
./newstext/smax/smax-6703462.txt.wakati
./newstext/smax/smax-6774314.txt.wakati
./newstext/smax/smax-6586549.txt.wakati
./newstext/smax/smax-6570795.txt.wakati
./newstext/smax/smax-6759232.txt.wakati
./newstext/smax/smax-6754338.txt.wakati
./newstext/smax/smax-6621762.txt.wakati
./newstext/smax/smax-6705948.txt.wakati
./newstext/smax/smax-6902433.txt.wakati
./newstext/smax/smax-6604610.txt.wakati


./newstext/smax/smax-6628113.txt.wakati
./newstext/smax/smax-6785403.txt.wakati
./newstext/smax/smax-6569160.txt.wakati
./newstext/smax/smax-6550978.txt.wakati
./newstext/smax/smax-6788602.txt.wakati
./newstext/smax/smax-6543509.txt.wakati
./newstext/smax/smax-6749426.txt.wakati
./newstext/smax/smax-6741327.txt.wakati
./newstext/smax/smax-6588884.txt.wakati
./newstext/smax/smax-6611266.txt.wakati
./newstext/smax/smax-6682299.txt.wakati
./newstext/smax/smax-6569288.txt.wakati
./newstext/smax/smax-6830704.txt.wakati
./newstext/smax/smax-6595584.txt.wakati
./newstext/smax/smax-6770787.txt.wakati
./newstext/smax/smax-6788401.txt.wakati
./newstext/smax/smax-6709919.txt.wakati
./newstext/smax/smax-6599204.txt.wakati
./newstext/smax/smax-6601757.txt.wakati
./newstext/smax/smax-6792662.txt.wakati
./newstext/smax/smax-6775880.txt.wakati
./newstext/smax/smax-6563840.txt.wakati
./newstext/smax/smax-6730015.txt.wakati
./newstext/smax/smax-6648738.txt.wakati
./newstext/smax/smax-6526832.txt.wakati


### MLPでテキストの分類をしよう

In [203]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils, to_categorical
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics
import json
import numpy as np

In [87]:
max_words = 67391 # 入力単語数
nb_classes = 10 # 9カテゴリを分類

In [68]:
batch_size = 64
nb_epoch = 20

In [191]:
# MLPのモデルを生成
def build_model():
    model = Sequential()
    model.add(Dense(512, input_dim=max_words))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [207]:
# データを読み込み
data = json.load(open("./newstext/data.json"))
# data = json.load(open("./newstext/data.json"))
X = data["X"] # テキストを表すデータ
Y = data["Y"] # カテゴリデータ

In [208]:
# 学習
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
Y_train = to_categorical(Y_train, nb_classes)
print(len(X_train), len(Y_train))
model = KerasClassifier(
    build_fn=build_model,
    epochs=nb_epoch,
    batch_size=batch_size,
    verbose=0)
model.fit(np.array(X_train), np.array(Y_train))

5525 5525


In [209]:
# 予測
y = model.predict(np.array(X_test))
ac_score = metrics.accuracy_score(Y_test, y)
cl_report = metrics.classification_report(Y_test, y)
print("正解率=", ac_score)
print("レポート=\n", cl_report)

正解率= 0.9419109663409337
レポート=
               precision    recall  f1-score   support

           0       0.93      1.00      0.96       205
           1       0.97      0.95      0.96       211
           2       0.96      0.96      0.96       250
           3       0.96      0.91      0.94       206
           4       0.89      0.85      0.87       116
           5       0.91      0.89      0.90       204
           6       0.91      0.99      0.95       221
           7       0.93      0.89      0.91       221
           9       0.99      0.99      0.99       208

    accuracy                           0.94      1842
   macro avg       0.94      0.94      0.94      1842
weighted avg       0.94      0.94      0.94      1842



# 文章の類似度をn-gramで調べよう

## 文章の類似度を調べることについて

インターネット上にはたくさんの資料があり、そこから引用や転用も多く行われている。そのため、2つの文章が似ているかどうか、あるいは、関連があるのかどうかを調べたい場面も多くある。ここでは、レーベンシュタイン距離の計算や、n-gramを使って単語や文章の類似度を調べる方法を紹介する。

## レーベンシュタイン距離について

「レーベンシュタイン距離(Levenshtein distance)」とは、2つの文字列がどの程度異なっているかを示すものである。これは「編集距離(Edit Distance)」とも呼ばれている。スペルミスの修正や類似語句の検索などにも用いられている。また、バイオインフォマティクス分野やDNA配列同士の類似性を判断する際にも利用されている。  
例えば「カンヅメ」と「カキトメ」はどのくらい似ているのであろうか。レーベンシュタイン距離では、「カンヅメ」を「カキトメ」に編集するまでに、何回の文字列操作が必要になるかという点に注目して単語の距離を測る。すなわち、操作回数は2回であり、レーベンシュタイン距離も2となる。

## Pythonでレーベンシュタイン距離を計算するプログラム

In [6]:
# レーベンシュタイン距離を求める
def calc_distance(a, b):
    '''レーベンシュタイン距離を計算する'''
    if a == b:
        return 0
    a_len = len(a)
    b_len = len(b)
    if a == "":
        return b_len
    if b == "":
        return a_len
    # 二次元の表(a_len+1, b_len+1)を準備
    matrix = [[] for i in range(a_len+1)]
    for i in range(a_len+1): # 0で初期化
        matrix[i] = [0 for j in range(b_len+1)]
    # 0の時の初期値を設定
    for i in range(a_len+1):
        matrix[i][0] = i
    for j in range(b_len+1):
        matrix[0][j] = j
    # 表を埋める
    for i in range(1, a_len+1):
        ac = a[i-1]
        for j in range(1, b_len+1):
            bc = b[j-1]
            cost = 0 if (ac == bc) else 1
            matrix[i][j] = min([
                matrix[i-1][j] + 1, # 文字の挿入
                matrix[i][j-1] + 1, # 文字の削除
                matrix[i-1][j-1] + cost # 文字の置換
            ])
    return matrix[a_len][b_len]

In [7]:
# カキトメとカンヅメの距離は？
print(calc_distance("カキトメ", "カンヅメ"))

2


In [9]:
# 実行例
samples = ["イカダ", "イカスミ", "イカ", "サカナ", "サンマ", "カナダ"]
base = samples[0]
r = sorted(samples,
           key = lambda n: calc_distance(base, n))
for n in r:
    print(calc_distance(base, n), n)

0 イカダ
1 イカ
2 イカスミ
2 サカナ
2 カナダ
3 サンマ


## N-gramで類似度を調べよう

文章の類似度を調べる方法には、ほかに「N-gram」を利用するものがある。  
「N-gram」とは、テキストで「隣り合ったN文字」のことを指している。異なる2つの文章のN-gramを総当たりで比較するなら、文節の順番が異なっていようとも、出現する単語の種類や頻度を比較することができる。これを利用して、論文の丸写しや盗用・コピペ、文章のリライトなどを検出することができる。

In [11]:
def ngram(s, num):
    # 隣り合うnum文字で区切った文章をresにappendしていく。
    res = []
    slen = len(s) - num + 1
    for i in range(slen):
        ss = s[i:i+num]
        res.append(ss)
    return res

In [12]:
def diff_ngram(sa, sb, num):
    a = ngram(sa, num)
    b = ngram(sb, num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                r.append(i)
    return cnt/len(a), r

In [13]:
a = "今日、渋谷で美味しいトンカツを食べた。"
b = "渋谷で食べた今日の豚カツは美味しかった。"

In [14]:
# 2-gram
r2, word2 = diff_ngram(a, b, 2)
print("2-gram:", r2, word2)

2-gram: 0.5 ['今日', '渋谷', '谷で', '美味', '味し', 'カツ', '食べ', 'べた', 'た。']


In [15]:
# 3-gram
r3, word3 = diff_ngram(a, b, 3)
print("3-gram:", r3, word3)

3-gram: 0.17647058823529413 ['渋谷で', '美味し', '食べた']


# マルコフ連鎖やLSTMで文章を生成しよう

## マルコフ連鎖とLSTM/RNNについて

本節では、文章の自動生成に挑戦する。マルコフ連鎖とLSTM/RNNは、それぞれ、異なるアプローチによる文章生成手法。マルコフ連鎖は、確率過程を使って文章をつなぎ合わせていく手法で、LSTM/RNNは機械学習によって、次にクルボジを予測するという手法で文章を生成する。

## マルコフ連鎖とは

マルコフ連鎖とは、確立過程の一種である。これは、ロシアの数学者マルコフによって研究されたもので、物理学や統計学の基本的なモデルに応用されている。マルコフ連鎖を利用すると、既存の文章をもとにして、自動で文章を生成することができる。ワードサラダとも呼ばれている。ここでは、難しい事は抜きにして、マルコフ連鎖を利用して、文章生成に挑戦する。

マルコフ性(Markov property)とは、次の状態が過去の状態に依存せず現在の状態のみによって決まる性質のことをいう。マルコフ性が存在する場合、状態が{q0, q1, q2, q3, ...q(n-1)}と、n通りの状態をとりえる状態を考える。現在の状態がqiであった時に、次の状態qjへ遷移する確率は、次の状態と現在の状態のみで記述し、P(qj|qi)で決定する。  
これを利用すると、仕組みが単純でありながら、面白い文章を生成してくれる。  
また、マルコフ連鎖は文章の要約という側面も持っている。機械的に文章を生成し、それをTwitterに自動で発言するようなボットのプログラムにも利用される。  
作文の仕組みを箇条書きにすると以下のようになる。  
1. 文章を単語に文化(形態素解析)する。
2. 単語の前後の結びつきを辞書に登録する。
3. 辞書を利用してランダムに作文する  
この辞書の作り方に特徴があり、分かち書きした文章を、前後の結びつきで登録していく。例えば、「彼は猫が好きだ」等文章があるならば、「彼|は|猫|が|好き|だ」と文章を分割する。そしてこれを単語の前後に注目して、3要素ごとに辞書へ登録する。辞書は次のように登録する。  

彼|は|猫  
は|猫|が  
猫|が|好き  
が|好き|だ  

つまり、前項で紹介したn-gramを文字単位ではなく、単語単位で行うものである。  
ここでは、以下の猫に関することわざを辞書として与えたとする。この辞書をもとにして同じ組み合わせを持つ単語と単語をランダムに組み合わせて文章を生成する。  

猫|に|小判|。  
魚|を|猫|に|預ける|。  
窮鼠|猫|を|嚙む|。  
預ける|猫|も|鰹節|。  

例えば、文頭に「猫」を使うとしたら、猫に繋がる要素は「猫|に」「猫|を」「猫|も」。ここでは「に」を選ぶ。そして、「に」につながる要素は、「に|小判」「に|預ける」である。　　
ただし、マルコフ連鎖では前後の単語の関連性が薄く、意味を考えて選ぶわけで絵はないため、出鱈目な文章になることが多い。

In [19]:
from janome.tokenizer import Tokenizer
import os, re, json, random

In [55]:
# マルコフ連鎖の辞書を作成
def make_dic(words):
    tmp = ["@"]
    dic = {}
    for i in words:
        word = i.surface
        if word == "" or word == "\r\n" or word == "\n":
            continue
        tmp.append(word)
        if len(tmp) < 3: # tmpが3文字以上になるようにする
            continue
        if len(tmp) > 3: # tmpが4文字になったら3文字に減らす
            tmp = tmp[1:]
        set_word3(dic, tmp) # 辞書に登録
        if word == "。": # 句読点ができたら、初期値@にする
            tmp = ["@"]
            continue
    return dic

In [56]:
# 三要素のリストを辞書として登録
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic:
        dic[w1] = {}
    if not w2 in dic[w1]:
        dic[w1][w2] = {}
    if not w3 in dic[w1][w2]:
        dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1

In [57]:
# 作文する
def make_sentence(dic):
    ret = []
    if not "@" in dic:
        return "no dic"
    top = dic["@"]
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True:
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == "。":
            break
        w1, w2 = w2, w3
    return "".join(ret)

In [61]:
def word_choice(sel):
    keys = sel.keys()
    return random.choice(list(keys))

In [59]:
# 文章を読み込む
sjis_file = "kokoro.txt.sjis"
dict_file = "markov-kokoro.json"
if not os.path.exists(dict_file):
    # Shift_JISの青空文庫のテキストを読み込む
    sjis = open(sjis_file, 'rb').read()
    text = sjis.decode('shift_jis')
    # 不要な部分を削除する
    text = re.split(r'\-{5,}', text)[2] # ヘッダーを削除
    text = re.split(r'底本:', text)[0] # フッターを削除
    text = text.strip()
    text = text.replace('|', '') # ルビの開始記号を削除
    text = re.sub(r'《.+》', '', text) # ルビを削除
    text = re.sub(r'[#.+?]', '', text) # 入力注を削除
    # janomeで形態素解析
    t = Tokenizer()
    words = t.tokenize(text)
    # 辞書を生成
    dic = make_dic(words)
    json.dump(dic, open(dict_file, "w", encoding="utf-8"))
else:
    dic = json.load(open(dict_file, "r"))

In [64]:
# 作文
for i in range(3):
    s = make_sentence(dic)
    print(s)
    print("---")

幸いにＫと並んで足を向けた人の結婚問題について、人に共通な興味のある先生を幸福にできるものはないんですがもっと真中へ出たり何かを先生にかぶれたんじゃありませんね。
---
奥さんの語気は前からの約束であったらしい。
---
せっかく来た主意が立たなくなるだけです。
---


In [69]:
from pprint import pprint
import json

In [71]:
dic = json.load(open("markov-kokoro.json"))
pprint(dic["歩き"])

{'ながら': {'の': 1}, '出し': {'た': 4, 'まし': 1, 'ます': 1}, '廻し': {'まし': 1}}


## LSTM/RNNについて

マルコフ連鎖と同じように文章を生成するアルゴリズムで有名なものに、再起型ニューラルネットワーク(Recurrent Neural Network/ RNN)や、LSTM(Long Short Term- Memory)がある。  RNNは、ニューラルネットワークを再起的に扱えるようにし、時系列モデルの解析ができるようにしたものであり、LSTMはRNNを改良したもので、RNNが多階層になりすぎると過去の依存関係を覚えきれないので、RNNに長期的に情報を記憶するためにブロックを採用したものである。そもそも、いろいろなデータを見ていくと「ABC」ときたら、次に「D」が来る可能性が高いというように、前のデータが次データと関係しているものがある。ニューラルネットワークに、前に発生したデータを投入できるようにしたものがRNNである。  時系列データが扱えると便利なのが文章の生成である。「今日の」という入力があった時、そこには「私」とか「天気」などがつながることが予想できる。そして、さらに「今日の天気」を入力すると、「は雨」とか「は晴れ」など、次々と文章を生成することができる。

### LSTMで文章を生成する

In [74]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys

In [75]:
# 元となるテキストを読み込む
path = "./text/夏目漱石/我輩は猫である.txt"
bindata = open(path, "rb").read()
text = bindata.decode("shift_jis")
print('コーパスの長さ:', len(text))

コーパスの長さ: 377325


In [76]:
# 一文字ずつバラバラにして文字にIDを振る
chars = sorted(list(set(text)))
print('使われている文字の数:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # 文字 -> ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID -> 文字

使われている文字の数: 3104


In [77]:
# テキストをmaxlen文字で区切って、その分の次に来る文字を記録する
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
print('学習する分の数：', len(sentences))

学習する分の数： 125769


In [81]:
print('テキストをIDベクトルにします...')
# X[i, j, k]: 20文字で区切られた文章のi番目の文のj番目の文字に文字kがある
# y[i, k']: 20文字で区切られた文章のi番目の文に対して、次に来る文字を設定する
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences): # i: 文章の中にある文のインデックス
    for t, char in enumerate(sentence): # t: 文の中にある文字のインデックス
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

テキストをIDベクトルにします...


In [84]:
# モデルを構築する(LSTM)
print('モデルを構築します...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

モデルを構築します...


In [85]:
# 選択候補となる配列から値を取り出す
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [88]:
# 学習させて、テキストを生成する・・・を繰り返す
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('繰り返し=', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # 訓練
    # ランダムにテキストのシードを選ぶ
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 多様性のパラメータごとに文を生成する
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('---多様性---', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('---シード="' + sentence + '"')
        sys.stdout.write(generated)
        # シードをもとにテキストを自動で生成する
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 次に来る文字を予測
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 既存の文に予測した一文字を足す
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
繰り返し= 1
Epoch 1/1


<ipython-input-88-d9cf1c4ead8e>:6: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  model.fit(X, y, batch_size=128, nb_epoch=1) # 訓練


125769/125769 [==============================] - 246s 2ms/step - loss: 3.7892

---多様性--- 0.2
---シード="1月、2月、4月、6月、7月、10月
"
1月、2月、4月、6月、7月、10月
「ええええ、ただ、どうしても知らない。
「そんな事をしているのだろうと云うと、どうしてもそのうちにはない。
「そんな事をしているのです」
「それから、これからですよ」
「そんな事をしているのだから、あのためにはない。
「そんな事をすると云うとこのにはなるほどの方があるから、そのうちにはない。ただから、この人間がいつの間はまたはないと云ったら、そのうちにはないと云ったら、そのうちにはない。
「そんな事をしている。
「そんな事をしている。
「そんな事をしている。
「そんな事をしているのは、このこの間はこの間は人間の方へ出している。
「それでもしからない。
「そんな事をしている。
「そんな事をしているのは、このこの中にはずらずと云うのは、このうちにはこの人間はこの間の方へ出ているのだから、そのそのうちにはいいのは、あの方があるから、そのうちはこの人間の方へ出ている。
「

---多様性--- 0.5
---シード="1月、2月、4月、6月、7月、10月
"
1月、2月、4月、6月、7月、10月
するかかように這入《はい》っているところは御いさんだって、どうしてもってるのです」「あやがい」
「そんな事をしている。せっかしているのは、何ともうよくってしたのかも知れない。
「それから、そのようにもっても、しかしかしかして見ただって、このいつだけはいいのに、その人間のごとくのこの、また｜《おお》っている。
「それから、人間の月になると云ってはいいになる。
「あなたのところですから、あのりのめてもう人にはいつの間《ま》になると、しかしそのように主人がているのから、このほどのようになるかと云うと、するとしているのはない。せんや、いつが事をしているのかくらんです」
「そのかい」
「そのかい」と迷亭のでも見える。
「あれまえた」
「ただから」
「何でもない。
「そんな事をしているのは、この
「そんな事を出して、そのくらいのあと、どうものですから、どうですよ」
「なるほどの

---多様性--- 1.0
---シ

彼等は入らぬ算段をして種々雑多な恰好《かっこうかい》になって、その時の間からこの人間がない。それから、その時分はない。それだから僕はその時分の間から聞いている。「それではないか、それではないから、それから、その時の間からのはない。それはどうしてもいいから、それから、その時分はない。それから、その時の間から、その中《うち》のところが、その時の間から、このものはない。それだから何と云うものはない。それだから、その時の間はこの時間の間《ま》にからから、そうして、その時の間から何の事ですが、どうしてもいいから、そうして、その時の間からのはない。
「そりゃ、そうして、その上になった。それではない。それだから、それから、それでもない。それだから、その時分はないと云う。「あのためになったから、これをかたまえている。
「そりゃ、そりゃ何がない。それではない。それでもない。
「それはこれをかたいたのは、これからないのは、その時は大《おおい》

---多様性--- 0.5
---シード="彼等は入らぬ算段をして種々雑多な恰好《か"
彼等は入らぬ算段をして種々雑多な恰好《かっこうかい》になって、すると云うと、吾輩はこの時間をもっても、何だかっても、これでもない。
「どうも、それじゃ、そりゃ、そうして、これを一人《いんきん》のように。
「それじゃありませんから」と云う。
「うん、いくら顔を出して、そうしてこのならないから、僕は先生の間《ま》にはない。主人もありませんから、今日《こんにち》の事ですが、何でもありますが、一
「どうも御のせんなったのだから、どうも、どうしてもいいから、ところがりさ」と云うと、その時の人間になったが、大きないのか、それから、何と云わない。
「そうます、その子がいい。それだから、そうしては年聞からしい。こうしてもらいたのはものと、これは金田のとへへへその中にかいている。
「寒月君は大分《だいぶ》るためにも、いたのはいい。それでもまたものかないのは、ただそうなら、どうか、僕が間から御の間《ま》にからからない。それからずにかんと

---多様性--- 1.0
---シード="彼等は入らぬ算段をして種々雑多な恰好《か"
彼等は入らぬ算段をして種々雑多な恰好《かっばくだいり》ばかりと云ごをず開余る時的も僕が入れないと今までもまだ同。その時そのなしの間《ま》に高屋へ年隣食うだ

「いやになったものか」
「それではなかなかなかなかなかなかなかなかったと思いますと云うと云うと、この時は主人の方へも何とか云うのは、どうもこれを見たまえ事がある。
「それではなかなかなかなかなかなかなかなかなかなかろうと思いますと云うと、この時は吾輩の方へはある。
「それでもまた相手にしても、どうもこれを見ると、この人間はこの一人にはならない。
「いや、それだから、それでもまた出来ない。
「そうだな。いかなるかも知れない。
「そ

---多様性--- 0.5
---シード="そろ》った所だと少々驚いた。金田君はなお"
そろ》った所だと少々驚いた。金田君はなおと云い、その
「うんだって、どうもこれを本当にね、僕はどこにもなんですか」
「そうだ。この時にはいつの間《ま》にかくのものは主人が出来ている。
「それじゃ、そうならんと教えてやる、どうもこれを驚ろかたが後《うし》ろへ、おかしくの云うのは不思議な事もない。
「何だなと思うと、これはどうもこれだからなくなったが、やがんですよ」
「そうですか」
「いえ、どうも、少しもうなものなら、これを来たんですか」
「いいえ、それだからない。天下にあるたしかにはないから、少しは吾輩をいやになったような事を云うと、猫のはいけになると見えて、やったり、どうも何の事でも行かない。
「うんなんぞさ」
「そうにはなかなかな」
「うん」
「そうですか」と主人は気が一人がない。
「そんなになったら、この時は今日《こんにち》は分らんが、とうていはいやになったのかも知れない。
　これではどうしても、時

---多様性--- 1.0
---シード="そろ》った所だと少々驚いた。金田君はなお"
そろ》った所だと少々驚いた。金田君はなお通り――あらまた容はないつにかんと下さい。よかさ。頭がやりましたに」
「どうもこれでもあると問いね。もしくなった」
「で恋ラ代の平来ませんぼこうり付けんかいやが、敵のこれを一間きは寒廻をする。いまえ。ところがスーと感新すから「何もどん》首白切なるって、本当に考えると主人がきかのばかり無かい。するとされるのは吾輩の結論の事を道々とすればいい。質普通のなるそんなものに動いと云うはなかなかあらはしいゃないだ。この大、おやりだかっためて男よ」「まあ、先生は十な十――一くなかなか［＃「頭みでにかく、一心の物に相とに立んたかと二立た。も

...

「そいつは少々失敬した。それだから探偵さえはすに通じ《そうせん》や空時から安《とすう》る。はいけ僕に苦沙弥《うし》から首縊りのいり出してさ事が出着口を見る。その木上のが手方質でではおん。たい。の手このtちょうむ［＃「かも」は学校の大丈夫だも大《おおい》にかわして、、さあふましたくなるえ。御中へ珍る訳す［＃「なのかね」に傍点］と見《つわい》くする。
「いうも構よじゅうじゃありました」
「私の事じゃ本うに美学勢なね」と云うもちといかなは口を落雲館のこの張り［＃これざを出しきの由《ま》ざった時をのる人間ではあるったんだからな」「法はとも、やみに違ない。金田かだれぎ」と文で長、合わない事になるべくは分方大方《かっかいんちゃくりのする、いつと水して迷亭先生のあと己《おろ》れわざぬ」＃第家物ならも入似てい［＃「ずつたりは」に傍点］が人間が張りるる。
「後《のく》に敬るに気を切うかけまあ、無事に君者だと云いねん。そうし名が生

--------------------------------------------------
繰り返し= 50
Epoch 1/1
125769/125769 [==============================] - 223s 2ms/step - loss: 2.3492

---多様性--- 0.2
---シード="大抵三口半か四口で食うんですね。それより"
大抵三口半か四口で食うんですね。それよりはいいでいたと思う。
「そんな事はどうです」
「そんな事はどうだい」
「そんな事はどうして独仙君、彼等君だから、人の御前から、人間はありません」と主人が云うと思って、その間《ま》にしってからとまた思うとある人間はこの間にもなる。彼の面白いところを見ると、何でも、そんな事を云うと思いながら、その時分はとりかと思ったよ。ねえ今日はあるものだが、もっともないようにここにあまり出します。
「そんな事はどうです」
「そんな事はどうですか、あなたに御気の毒だわね」
「それでも、そんな事を云うと思う。
「そんなところである。これはどうしたら、それでもそんなにはどこから飛び出したのですか、あの気の毒なのはどうしたんじゃない、そんな事であろう。そんな事はどうしたら、それでもいいです」
「そんな事はどうですか、あんなにはちょっと、これはいつでもいいです」「そ

しくない。鼠としても大変用心深い鼠である。いつきたのもいても真の金が客で［＃「しゅ」「説本の傍理をはふたた-」にとかと云ってある。とればまくいつませた事事だ。よくはわん、しち問題でへ行って見せめや女が気であるわ、やがろう、飛びも寒月と毒のわく、右のゃなは生きう。人の君に大き過ぎぬの猫でこのあず黒のつくついたものだと寒月君がヴァイオきましている。当人だも飲んでため、これだけはそんな［＃「また月」は何だしいけにもトんざ御明瞭にはむじゃしい。誰も来《なにぼおくのじめんぶかくべき所《としら》しくなくなか、御理年さ山聞代わないんだも吾輩は、ござんちゃい」
「御いよございは教師かじ答くんを両立《つっがくろ》だからたきにばか廻《せと》の考う訳なければ好きと全じつけたので一人容《くだとら》にこの想像聞く書斎から木《やづて》をでゅ分のけれがまつス舌の大まだないと云うす事がよく智識の存外に落ちるのである。大れち果に金田面あらずな雑口でも小事の免を

--------------------------------------------------
繰り返し= 54
Epoch 1/1
125769/125769 [==============================] - 286s 2ms/step - loss: 2.3237

---多様性--- 0.2
---シード="礼を演じてはいけない。柳宗元《りゅうそう"
礼を演じてはいけない。柳宗元《りゅうそう》の事だから、それから、何でも、何でもないが、その時は今度はでであるから出るんだが、何でも大分《だいぶ》見たんだが、その時の人間を見せたが、吾輩

<ipython-input-85-212cec131aec>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds)/temperature


は一人も大きな声を出す。「それじゃ、ああ」と云う。
「そうです」
「そりゃ、その御三《おさん》がない。これで「ハハハハハハハハ」と云うと主人は何だか分らない。
　いは何だかと思っているのです」「ええ」と云う。
　吾輩は大《おおい》において「へえ、いくら成ですなら、ですよ」
「そりゃ、まだだよ」
「何だかないのですよ」
「あの方はええ」「ええ」と云う。
　主人は寒月君が何ともない。
　吾輩は主人の一人とだが、いかんでは主人のように相違ない。いくらいならばかりである。主人はこの時の人はこの時の大文には相手なし何《かた》につけたようなもので、大きな声を出す。「それでも、ああ、まあ何かなかなかったかい」と主人はどうしたら、いかにもしがにこ

---多様性--- 0.5
---シード="礼を演じてはいけない。柳宗元《りゅうそう"
礼を演じてはいけない。柳宗元《りゅうそう》の悪口なくらいである。――何かにこの人間といったので、その時は今日はや、またどこそれが見ると主人は、どうしても分じているのか。そこさんがためにはもしなうなら、このみなら下り込んで云うと、主人は見れば、さすがらいいのですか、あなただとまだ出るのだから」
「そりゃえい」と主人は何だか思うんです」
「それもいろいろだよ」と迷亭はまじておいたのは、何とかと思ったら、これは何でもないですよ」
「そうしたら、それは自分である。大きな声を食う。「なるにひままさんと云うのです」
「坊ばはは分りかい」
「そりゃ、その大きな声でするのだから、それからどうも行って見たら、主人はもしころである。こんな頭をうんて五分《やだ》より向いてもの、物は食いというのである。少々｜何《》かても切りませんと主人は大《おおい》には立てた。
　この人間の一人に話したい。寒月君は何だか思われて見ると、――それからならない

---多様性--- 1.0
---シード="礼を演じてはいけない。柳宗元《りゅうそう"
礼を演じてはいけない。柳宗元《りゅうそう》で笑う声が何かな顔をした。何ですが、「ある君に火を出しるんですとあんな馬生探気にあるのだからせにいめに相第iiりした頃まですかろう。口に長くわるまいやっともっござる者は出得しい。吾輩もみなになった。
　せっかくの亡《くん》、その時動がこんなんだい。貰いをっえた者は第一十五切れないや手はう。主人はやがないそう

「何だいきっともうんだから、それでもいいから、かくのは大きなどをえて、あの「だから、君には一人も何と云われたり、またはいけないから見えず、吾輩の方は小供が見える。
「何だかないのですか」「それは

---多様性--- 0.5
---シード="ってあれだけの器量なら、どこへだって相応"
ってあれだけの器量なら、どこへだって相応の男がついて、よくもとにかくところではない。断わりが、までは大きな食ったよ」「日は中学臘から吾輩が僕の面白いのに極《き》めます。
　吾輩が無理の事には吾輩のうちに、かくのくれる事には一人も出取った。もっともなる一人ときにおかしなたのだから、それでもまただが、ようやくから出来になってもいっても見えましょうか」と云う。
「何だいい、いけらい所でも、どうしてもいいからかい」と主人は大《おおい》に一《ひずつそ》している。しかしてはちょっとだから見えて、それが一人も十一日の鼻を食《おろ》になっていたが、これは面白いのです、
「それは無論の間、まる当って、どうしても動んだ弱先下じっているんだが、今日はその中から見た事のだ。一人が取っているのを極《き》めたって見たが、それから寒月君は天下来た事はこれでね」
「それが一人も知れないのですか。こうぞ。寒月君はどうして、一人も這入れますま」
「それが

---多様性--- 1.0
---シード="ってあれだけの器量なら、どこへだって相応"
ってあれだけの器量なら、どこへだって相応の顔にこのつけて見ればかいかなってながらい［＃訳せ込八一楽誘い五でやく。金田の出来《まきんやし》、今の地蔵に金作々覚えすと云う方がはかい」
「考え込んだのは直ちに武右がけて見たが買って来たの代りに見えつけるのだ。……」
［＃ここで字下げ］」、らば。い装ので向ぜかえ思うのは」「それは一りて日が意を成立るです［＃「かば」に傍点］よだね。それだから取ってはないかな」と御見えて」「
「無化物だ　ふとして足がながる心得てしまいました。それにははないそうがなれかに寒月｜重やと十だ。するようにつかみけなれば元気へ向うするばかりで、例のごとく間に出た。さっきにはなってくらず十一｜をころがって行く。
「一人やりならなかり見わるんです」
》そう十だ。「暗論をおろい。帰りはせんだが者は鈴木《ぬげのき込わのうちに真多いよ。……」
［＃ここまでに傍点］て見ろの事がある